# 3D model

We import the math libraries, data analysis libraries and image libraries. Also we set the figures and data dictionaries.

In [ ]:
# math libraries

import numpy as np
import sympy

from sympy import *
t = Symbol('t')

from scipy.interpolate import interp2d
from scipy.interpolate import griddata

import math

from skimage.measure import  points_in_poly

# data analysis library


import pandas as pd

from copernico import *
import utm


# import plotting libraries

import plotly.offline as go_offline
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as ticker

# imaging library

from PIL import Image

DATADIR='data/' # Directory with the data
FIGURESDIR='figures/' # Figures produced

We prepare the topography data

In [ ]:
# Topography data

topo=pd.read_csv(DATADIR+'topografy.csv')

#We split the data in topo to get three numpy arrays

topo1=topo[['UTM_X','UTM_Y','elevation']].to_numpy()

topox=[x[0] for x in topo1]
topoy=[x[1] for x in topo1]
topoelv=[x[2] for x in topo1]

# In the list cotasxy we save the dimensions of the model.

cotasxy=[min(topox)+200,max(topox)-2000,min(topoy)+100,max(topoy)-200]

The following function <span style="color:blue">surface</span> interpolates the data on a grid of points, the interpolation method can be linear, cubic or nearest.

In [ ]:
def surface(grid,metodo): #metodo='linear', 'cubic' o 'nearest'

    tx=grid[0]
    ty=grid[1]
    tz=grid[2]

    X = np.linspace(min(tx), max(tx))
    Y = np.linspace(min(ty), max(ty))
    X, Y = np.meshgrid(X, Y)  # 2D grid for interpolation
    Z = griddata((tx,ty),tz,(X,Y), 
                 method=metodo)
    
    return [tx,ty,tz,X,Y,Z]

We choose the linear interpolation method and apply it to the topographic data.

In [ ]:
topo_gr=[topox,topoy,topoelv]
topo_linear=surface(topo_gr,'linear')

We are ready to create the figure that realizes the 3D model. We use two color scales 'YlGn' and 'gray' to get more depth in the model.

In [ ]:
fig=go.Figure()

# topografía

fig.add_trace(go.Surface(  x=topo_linear[3],y=topo_linear[4],z=topo_linear[5],
                           colorscale='brwnyl',
                           opacity = 0.8,
                           name='Topography',
                           legendgroup='topo',
                           showlegend=True,
                           showscale=False))
fig.add_trace(go.Surface(  x=topo_linear[3],y=topo_linear[4],z=topo_linear[5],
                           colorscale='gray',
                           opacity = 0.3,
                           name='Topo',
                           legendgroup='topo',
                           showlegend=False,
                           showscale=False))

camera = dict(up=dict(x=0, y=0, z=1),
              center=dict(x=0, y=0, z=0),
               eye=dict(x=0, y=-1, z=2) )

fig.update_layout( title="Topography Model",
                  #paper_bgcolor = 'black',
                 scene = dict(
                     xaxis=dict(title='UTM_X', 
                                tickfont = dict(size = 10,color = 'black'),
                                title_font_size=10,
                                titlefont_color='black', 
                                range=[cotasxy[0],cotasxy[1]],
                                backgroundcolor='white',
                                color='black',
                                gridcolor='gray'),
                     yaxis=dict(title='UTM_Y',
                                tickfont = dict(size = 10,color = 'black'),
                                title_font_size=10,
                                titlefont_color='black', 
                                range=[cotasxy[2],cotasxy[3]],  
                                backgroundcolor='white',
                                color='black',
                                gridcolor='gray'),
                      zaxis=dict(nticks=4,
                                tickfont = dict(size = 10,color = 'black'),
                                title='Elevation', 
                                title_font_size=10,
                                titlefont_color='black', 
                                range=[500,1000],
                                backgroundcolor='white',
                                color='black', 
                                gridcolor='gray'),
                     aspectratio=dict(x=2, y=1.8, z=0.3)),
                     #aspectmode='data'),
                     scene_camera= camera
                 ) 
              

fig.show()

We read and process the data for contacts

In [ ]:
def contact_tr(csv,elevation_gain):
    tr=pd.read_csv(csv)
    tr1=tr[['UTM_X','UTM_Y','elevation']].to_numpy()
    tx=[x[0] for x in tr1]
    ty=[x[1] for x in tr1]
    telv=[x[2]+elevation_gain for x in tr1]
    return [tx,ty,telv]

# mechanic contacts

palomeque=contact_tr(DATADIR+'palomeque.csv',0)+['palomeque_thrust']
palomeque_ds=contact_tr(DATADIR+'palomeque_ds.csv',0)+['palomeque_desplazado']
calvillo=contact_tr(DATADIR+'calvillo.csv',0)+['calvillo_thrust']
calvillo_ds=contact_tr(DATADIR+'calvillo_ds.csv',0)+['calvillo_desplazado']

# faults

fault2=contact_tr(DATADIR+'fault2.csv',0)+['fault2']
fault1=contact_tr(DATADIR+'fault1.csv',0)+['fault1']
fault3=contact_tr(DATADIR+'fault3.csv',0)+['fault1']

# stratigraphic contacts

E12cal=contact_tr(DATADIR+'E1E2 calvillo.csv',0)+['E1-E2 calvillo']
E12pal=contact_tr(DATADIR+'E1E2 palomeque.csv',0)+['E1-E2 palomeque']
E23pal=contact_tr(DATADIR+'E2E3 palomeque.csv',0)+['E2-E3 palomeque']
E3Opal=contact_tr(DATADIR+'E3O palomeque.csv',0)+['E3-O palomeque']
PEpal=contact_tr(DATADIR+'PE palomeque.csv',0)+['P-E palomeque']
PEcal=contact_tr(DATADIR+'PE calvillo.csv',0)+['P-E calvillo']
E12cal_ds=contact_tr(DATADIR+'E1E2 calvillo_ds.csv',0)+['E1-E2 calvillo ds']
E12pal_ds=contact_tr(DATADIR+'E1E2 palomeque_ds.csv',0)+['E1-E2 palomeque ds']

The following function <span style="color:blue">contact_dat</span> uses the comad Scatter3d to 
create de contact data for the figure.

In [ ]:
def contact_dat(x,y,elv,h,mode,nam,color,gr,t,ls):
    z=[x+h for x in elv]
    return go.Scatter3d(x=x, y=y, z=z,
            mode =mode,
            name=nam,
            legendgroup=gr,
            showlegend=t,
            line=dict(color=color,
                      dash=ls,
                      width=5)
                        )

In [ ]:
def falls(i,falla,s,h,r,rr,dd):
    A=(falla[0][i],falla[1][i])
    c1z=falla[2][i]+h
    c2x=falla[0][i+1]
    c2y=falla[1][i+1]
    d=(c2x-A[0],c2y-A[1])
    nd=np.sqrt(d[0]**2+d[1]**2)
    dn=(d[0]/nd,d[1]/nd)
    A1=(A[0]+dn[0]*s,A[1]+dn[1]*s-dd)
    B=(A1[0]+dn[0]*250,A1[1]+dn[1]*250)
    C=(B[0]-r,B[1]+rr)
    return [[A1[0],B[0],C[0]],[A1[1],B[1],C[1]],[c1z,c1z,c1z]]

In [ ]:
cont=[palomeque_ds,palomeque,calvillo,calvillo_ds]
cab=[E12cal,E12pal,E23pal,E3Opal,PEpal,PEcal,E12cal_ds,E12pal_ds]
fal=[fault1,fault2,fault3]

contacts=[contact_dat(palomeque_ds[0],palomeque_ds[1],palomeque_ds[2],30,
                        "lines",'Thrusts','black','contacts',True,None)
          ]+[contact_dat(x[0],x[1],x[2],30,"lines",x[3],'black','contacts',False,None) for x in cont[1:]]

cabs=[contact_dat(E12cal[0],E12cal[1],E12cal[2],30,"lines",'Stratigraphic contacts and units','black',
                            'cabalgamientos',True,'dot')
              ]+[contact_dat(x[0],x[1],x[2],30,"lines",x[3],'black','cabalgamientos',False,'dot') for x in cab[1:]]

faults=[contact_dat(fault1[0],fault1[1],fault1[2],50,"lines",'Strike-slip faults','black','faults',True,None)
      ]+[contact_dat(x[0],x[1],x[2],30,"lines",x[3],'black','faults',False,None) for x in fal[1:]]

f1=falls(1,fault1,300,0,50,0,40)
f2=falls(0,fault2,-400,40,60,20,100)

We add geological elements to the model.

In [ ]:
fig=go.Figure(contacts+cabs+faults)

# topography

fig.add_trace(go.Surface(  x=topo_linear[3],y=topo_linear[4],z=topo_linear[5],
                           colorscale='brwnyl',
                           opacity = 0.8,
                           name='Topography',
                           legendgroup='topo',
                           showlegend=True,
                           showscale=False))
fig.add_trace(go.Surface(  x=topo_linear[3],y=topo_linear[4],z=topo_linear[5],
                           colorscale='gray',
                           opacity = 0.3,
                           name='Topo',
                           legendgroup='topo',
                           showlegend=False,
                           showscale=False))

camera = dict(up=dict(x=0, y=0, z=1),
              center=dict(x=0, y=0, z=0),
               eye=dict(x=0, y=-1, z=2) )

fig.update_layout( title="Topography Model",
                  #paper_bgcolor = 'black',
                 scene = dict(
                     xaxis=dict(title='UTM_X', 
                                tickfont = dict(size = 10,color = 'black'),
                                title_font_size=10,
                                titlefont_color='black', 
                                range=[cotasxy[0],cotasxy[1]],
                                backgroundcolor='white',
                                color='black',
                                gridcolor='gray'),
                     yaxis=dict(title='UTM_Y',
                                tickfont = dict(size = 10,color = 'black'),
                                title_font_size=10,
                                titlefont_color='black', 
                                range=[cotasxy[2],cotasxy[3]],  
                                backgroundcolor='white',
                                color='black',
                                gridcolor='gray'),
                      zaxis=dict(nticks=4,
                                tickfont = dict(size = 10,color = 'black'),
                                title='Elevation', 
                                title_font_size=10,
                                titlefont_color='black', 
                                range=[500,1000],
                                backgroundcolor='white',
                                color='black', 
                                gridcolor='gray'),
                     aspectratio=dict(x=2, y=1.8, z=0.3)),
                     #aspectmode='data'),
                     scene_camera= camera
                 ) 
              

fig.show()


We add texts for heights and Stratigraphic levels and other geological simbols to the figure 

In [ ]:
namesx=[619200,620800]
namesy=[4196500,4196250]
namesz=[850,850]
namestxt=['Calvillo height','Palomeque height','A','A$^\prime$']

tx=[calvillo[0][1]+50,calvillo[0][3],calvillo[0][4],calvillo[0][5],calvillo[0][7]]
ty=[calvillo[1][1]-50,calvillo[1][3]-50,calvillo[1][4]-70,calvillo[1][5]-50,calvillo[1][7]-70]
tz=[calvillo[2][1]+40,calvillo[2][3]+30,calvillo[2][4]+50,calvillo[2][5]+15,calvillo[2][7]+20]

nnx=[618900,619350,620200,619350,619400,620120,620700,
     618850,619200,620900,621100,619700]
nny=[4196800,4196700,4196400,4196200,4196000,4196100,4196000,
     4195500,4195500,4195800,4195500,4195200]
nnz=[700,750,750,750,700,790,700,750,700,600,650,700]
nntxt=['M1','P','P','E1','E2','E1','E2','E1','E2','E3','O2','E1']

def eq(A,B): # A and B are points
    x1, y1 = A
    x2, y2 = B
    # calculate the distance between the two points
    distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

    # calculate the angle between the two points
    angle = math.atan2(y2 - y1, x2 - x1)

    # calculate the coordinates of the third point (the first one)
    x3_1 = x1 + distance * math.cos(angle - (1 * math.pi / 3))
    y3_1 = y1 + distance * math.sin(angle - (1 * math.pi / 3))

    # calculate the coordinates of the third point (the second one)
    x3_2 = x1 + distance * math.cos(angle + (1 * math.pi / 3))
    y3_2 = y1 + distance * math.sin(angle + (1 * math.pi / 3))
    return[[x3_1,y3_1],[x3_2,y3_2]]

    

def tr(i,contacto,h,s,hh):
    A=(contacto[0][i],contacto[1][i])
    c1z=contacto[2][i]+h
    c2x=contacto[0][i+1]
    c2y=contacto[1][i+1]
    d=(c2x-A[0],c2y-A[1])
    nd=np.sqrt(d[0]**2+d[1]**2)
    dn=(d[0]/nd,d[1]/nd)
    B=(A[0]+dn[0]*150,A[1]+dn[1]*150)
    C=eq(A, B)[s]
    return [[A[0],B[0],C[0]],[A[1],B[1],C[1]],[c1z,c1z,c1z+hh]]

tcalvillo=[tr(1,calvillo,15,0,100),tr(3,calvillo,15,0,50),tr(5,calvillo,15,0,50),tr(7,calvillo,15,0,0)]

tcalvillo_dat=[go.Mesh3d(x=x[0],y=x[1],z=x[2],
                        alphahull=5, opacity=1, color='black',
                        i = np.array([0]),
                        j = np.array([1]),
                        k = np.array([2]),
                         legendgroup='contacts',
                         showlegend=False,
                       ) for x in tcalvillo]

tcalvillo_ds=[tr(1,calvillo_ds,25,1,30)]
tcalvillo_ds_dat=[go.Mesh3d(x=x[0],y=x[1],z=x[2],
                        alphahull=5, opacity=1, color='black',
                        i = np.array([0]),
                        j = np.array([1]),
                        k = np.array([2]),
                            legendgroup='contacts'
                       ) for x in tcalvillo_ds]

tpalomeque=[tr(1,palomeque,15,0,70),tr(4,palomeque,15,0,70),tr(5,palomeque,15,0,90),
            tr(6,palomeque,15,0,70),tr(7,palomeque,15,0,30)]

tpalomeque_dat=[go.Mesh3d(x=x[0],y=x[1],z=x[2],
                        alphahull=5, opacity=1, color='black',
                        i = np.array([0]),
                        j = np.array([1]),
                        k = np.array([2]),
                          legendgroup='contacts'
                       ) for x in tpalomeque]

tpalomeque_ds=[tr(1,palomeque_ds,5,1,0)]

tpalomeque_ds_dat=[go.Mesh3d(x=x[0],y=x[1],z=x[2],
                        alphahull=5, opacity=1, color='black',
                        i = np.array([0]),
                        j = np.array([1]),
                        k = np.array([2]), 
                        legendgroup='contacts',
                       ) for x in tpalomeque_ds]

tcont=tcalvillo_dat+tcalvillo_ds_dat+tpalomeque_dat+tpalomeque_ds_dat

In [ ]:
fig=go.Figure(contacts+cabs+faults+tcont)

# topography

fig.add_trace(go.Surface(  x=topo_linear[3],y=topo_linear[4],z=topo_linear[5],
                           colorscale='brwnyl',
                           opacity = 0.8,
                           name='Topography',
                           legendgroup='topo',
                           showlegend=True,
                           showscale=False))
fig.add_trace(go.Surface(  x=topo_linear[3],y=topo_linear[4],z=topo_linear[5],
                           colorscale='gray',
                           opacity = 0.3,
                           name='Topo',
                           legendgroup='topo',
                           showlegend=False,
                           showscale=False))


trace = go.Scatter3d(
                   x=namesx, y=namesy, z=namesz,
                   text=namestxt,
                   name='Heights',
                   mode="text",
                   textfont=dict(color=["black","black"],size=13),
                   hoverinfo="skip")

fig.add_trace(trace)

trace2 = go.Scatter3d(
                   x=nnx, y=nny, z=nnz,
                   text=nntxt,
                   name='Stratigraphic levels',
                   mode="text",
                   legendgroup='cabalgamientos',
                   showlegend=False,
                   textfont=dict(color='black',size=18),
                   hoverinfo="skip")

fig.add_trace(trace2)


camera = dict(up=dict(x=0, y=0, z=1),
              center=dict(x=0, y=0, z=0),
               eye=dict(x=0, y=-1, z=2) )


fig.update_layout( title="3D Palomeque sheets geological map",
                  #paper_bgcolor = 'black',
                 scene = dict(
                     xaxis=dict(title='UTM_X', 
                                tickfont = dict(size = 10,color = 'black'),
                                title_font_size=10,
                                titlefont_color='black', 
                                range=[cotasxy[0],cotasxy[1]],
                                backgroundcolor='white',
                                color='black',
                                gridcolor='gray'),
                     yaxis=dict(title='UTM_Y',
                                tickfont = dict(size = 10,color = 'black'),
                                title_font_size=10,
                                titlefont_color='black', 
                                range=[cotasxy[2],cotasxy[3]],  
                                backgroundcolor='white',
                                color='black',
                                gridcolor='gray'),
                      zaxis=dict(nticks=4,
                                tickfont = dict(size = 10,color = 'black'),
                                title='Elevation', 
                                title_font_size=10,
                                titlefont_color='black', 
                                range=[100,1500],
                                backgroundcolor='white',
                                color='black', 
                                gridcolor='gray'),
                     aspectratio=dict(x=2, y=1.8, z=1)),
                     #aspectmode='data'),
                     scene_camera= camera
                 ) 
              

fig.show()

go_offline.plot(fig,filename=FIGURESDIR+'Topomodel.html',validate=True, auto_open=False)


## Color change in topography

Now we are going to change the colors of the topography. We associate a color to each material we find on the surface. To do this we start by defining a denser grid covering the region, then we define polygons enclosing each type of material and fill the interior of each of these polygons with the corresponding color.

We already have the grid with the elevations (calculated with Copernicus) in a csv file, we read it and prepare the data

In [ ]:
NT=pd.read_csv(DATADIR+'TN_el.csv')
NT1=NT[['x','y','elevation']].to_numpy()

NTx=[x[0] for x in NT1]
NTy=[x[1] for x in NT1]
NTelv=[x[2] for x in NT1]

gTN=[NTx,NTy,NTelv]

Define the lists of points that determine the polygons 

In [ ]:
# Crossed sections

sections=pd.read_csv(DATADIR+'sections.csv')

sections1=sections[['UTM_X','UTM_Y','elevation','point']].to_numpy()
secx=[x[0] for x in sections1]
secy=[x[1] for x in sections1]
secelv=[x[2] for x in sections1]
secp=[x[3] for x in sections1]

# split the data
secx2=[(secx[i],secx[i+1]) for i in range(len(secx)-1)]
secy2=[(secy[i],secy[i+1]) for i in range(len(secx)-1)]
secelv2=[(secelv[i],secelv[i+1]) for i in range(len(secx)-1)]


points=[[(secx[i],secy[i],secelv[i]),secp[i]] for i in range(len(secx))]


In [ ]:
pol1=[calvillo[0][:-5]+PEcal[0][::-1],
      calvillo[1][:-5]+PEcal[1][::-1],
      calvillo[2][:-5]+PEcal[2][::-1]]

pol2=[PEcal[0]+[calvillo[0][6]]+E12cal[0][::-1]+[fault1[0][1],PEcal[0][0]],
      PEcal[1]+[calvillo[1][6]]+E12cal[1][::-1]+[fault1[1][1],PEcal[1][0]]]

pol3=[E12cal[0]+calvillo[0][7:]+palomeque[0][:6][::-1]+[E12cal[0][0]],
      E12cal[1]+calvillo[1][7:]+palomeque[1][:6][::-1]+[E12cal[1][0]]]

pol4=[palomeque[0][:-2]+PEpal[0][::-1]+[palomeque[0][0]],
      palomeque[1][:-2]+PEpal[1][::-1]+[palomeque[1][0]]]

pol5=[PEpal[0]+[palomeque[0][-1]]+E12pal[0][::-1]+fault1[0][-4:-2][::-1]+[PEpal[0][0]],
      PEpal[1]+[palomeque[1][-1]]+E12pal[1][::-1]+fault1[1][-4:-2][::-1]+[PEpal[1][0]]]

pol6=[palomeque_ds[0]+E12pal_ds[0]+[palomeque_ds[0][0]],
      palomeque_ds[1]+E12pal_ds[1]+[palomeque_ds[1][0]]]

pol7=[E23pal[0]+E3Opal[0][::-1]+[620505,620029,E23pal[0][0]],
      E23pal[1]+E3Opal[1][::-1]+[4194758,4194758,E23pal[1][0]]]

pol8=[calvillo_ds[0][::-1]+[fault2[0][-2]]+E12cal_ds[0][::-1]+[calvillo_ds[0][-1]],
      calvillo_ds[1][::-1]+[fault2[1][-2]]+E12cal_ds[1][::-1]+[calvillo_ds[1][-1]]]

pol9=[[secx[0]-10,fault3[0][-1]]+calvillo_ds[0][::-1]+calvillo[0]+palomeque[0][6:]+[621420,secx[0]-10],
      [secy[2],fault3[1][-1]]+calvillo_ds[1][::-1]+calvillo[1]+palomeque[1][6:]+[secy[2],secy[2]]]

pol10=[fault3[0][-3:-1][::-1]+E12cal_ds[0]+fault1[0][1:6]+E12pal_ds[0][::-1]+palomeque_ds[0]+E12pal[0]+E23pal[0][::-1]+[620029,618332,fault3[0][-1]],
       fault3[1][-3:-1][::-1]+E12cal_ds[1]+fault1[1][1:6]+E12pal_ds[1][::-1]+palomeque_ds[1]+E12pal[1]+E23pal[1][::-1]+[4194758,4194758,fault3[1][-1]]]

pol11=[[620505]+E3Opal[0]+[cotasxy[1]],
       [4194758]+E3Opal[1]+[cotasxy[2]]]

The function `fil-plo3` select the point of a grid that are inside a polygon

In [ ]:
def fil_pol3(poly,grid):
    xypoly=[[poly[0][i],poly[1][i]] for i in range(len(poly[0]))]
    X1=grid[0]
    Y1=grid[1]
    Z1=np.array(grid[2])
    xy1=[[X1[i],Y1[i]] for i in range(len(X1))]

    mask1=points_in_poly(xy1,xypoly)

    Z1[np.where(~mask1)] = 0  # to avoid importing numpy.ma

    xyz1=[[X1[i],Y1[i],Z1[i]] for i in range(len(X1)) if Z1[i]!=0]

    xx1=[x[0] for x in xyz1]
    yy1=[x[1] for x in xyz1]
    zz1=[x[2] for x in xyz1]
    return [xx1,yy1,zz1]


The funtion `scatter` use the plot function Scatter3d to prepare the data

In [ ]:
def scatter(datos,nombre,opc,c,s,group,t):
    return go.Scatter3d(x=datos[0],
               y=datos[1],
               z=datos[2], 
               name=nombre,
               legendgroup=group,
               showlegend=t,
               mode='markers',
               marker=dict(
               size=s,
               color=datos[2],                # set color to an array/list of desired values
               colorscale=[[0, c], [1,c]],   # choose a colorscale
               opacity=opc))

In [ ]:
pol_list=[pol1,pol2,pol3,pol4,pol5,pol6,pol7,pol8,pol9,pol10,pol11]
name_list=['P','E1','E2','E3','M1','O2','7','8','M1','10','Topography']
color_list=['#e6d7ff','#E97451','pink','#e6d7ff','#E97451','#C19A6B','#D1BEA8','#E97451','yellow','pink','#C19A6B']
filled=[fil_pol3(x,gTN) for x in pol_list]
boll=[False for i in range(len(pol_list)-1)]+[True]
p_dat=[scatter(filled[i],name_list[i],0.8,color_list[i],3,'ct',boll[i]) for i in range(len(pol_list))]

In [ ]:
fig=go.Figure(contacts+cabs+faults+tcont+p_dat)

# texts

trace = go.Scatter3d(
                   x=namesx, y=namesy, z=namesz,
                   text=namestxt,
                   name='Heights',
                   mode="text",
                   textfont=dict(color=["black","black"],size=13),
                   hoverinfo="skip")

fig.add_trace(trace)

trace2 = go.Scatter3d(
                   x=nnx, y=nny, z=nnz,
                   text=nntxt,
                   name='Stratigraphic levels',
                   mode="text",
                   legendgroup='cabalgamientos',
                   showlegend=False,
                   textfont=dict(color='black',size=18),
                   hoverinfo="skip")

fig.add_trace(trace2)
 
# geological symbos for faults

fig.add_trace(go.Scatter3d(x=f1[0],
                        y=f1[1],
                        z=f1[2],
                       mode ='lines',
                       name='Sections',
                      legendgroup='faults',
                      showlegend=False,
                      line=dict(color='black',
                      width=5)
                       ))
fig.add_trace(go.Scatter3d(x=f2[0],
                        y=f2[1],
                        z=f2[2],
                       mode ='lines',
                       name='Sections',
                      legendgroup='faults',
                      showlegend=False,
                      line=dict(color='black',
                      width=5)
                       ))


# figure layout

camera = dict(up=dict(x=0, y=0, z=1),
              center=dict(x=0, y=0, z=0),
               eye=dict(x=0, y=-1, z=2) )

fig.update_layout( title="3D Palomeque sheets geological map",
                  #paper_bgcolor = 'black',
                 scene = dict(
                     xaxis=dict(title='UTM_X', 
                                tickfont = dict(size = 10,color = 'black'),
                                title_font_size=10,
                                titlefont_color='black', 
                                range=[cotasxy[0],cotasxy[1]],
                                backgroundcolor='white',
                                color='black',
                                gridcolor='gray'),
                     yaxis=dict(title='UTM_Y',
                                tickfont = dict(size = 10,color = 'black'),
                                title_font_size=10,
                                titlefont_color='black', 
                                range=[cotasxy[2],cotasxy[3]],  
                                backgroundcolor='white',
                                color='black',
                                gridcolor='gray'),
                      zaxis=dict(nticks=4,
                                tickfont = dict(size = 10,color = 'black'),
                                title='Elevation', 
                                title_font_size=10,
                                titlefont_color='black', 
                                range=[100,1500],
                                backgroundcolor='white',
                                color='black', 
                                gridcolor='gray'),
                     aspectratio=dict(x=2, y=1.8, z=1)),
                     #aspectmode='data'),
                     scene_camera= camera
                 ) 
              

fig.show()
go_offline.plot(fig,filename=FIGURESDIR+'geomap.html',validate=True, auto_open=False)


We are going to outline the limits of the topography

In [ ]:
punto1=(min(topox)+200+1,min(topoy)+100+1,700)
punto2=(min(topox)+200+1,max(topoy)-200-1,700)
punto3=(max(topox)-2000-1,max(topoy)-201,700)
punto31=(max(topox)-2000-1,max(topoy)-150,700)
punto4=(max(topox)-2000-1,min(topoy)+100+1,700)

xp=[min(topox),min(topox),max(topox),max(topox)]
yp=[min(topoy),max(topoy),max(topoy),min(topoy)]
zp=[700,700,700,700]

#  parametric equation of the line determined by two points

def para_line(p1,p2):
    P1=np.array(p1)
    P2=np.array(p2)
    v = P2 - P1
    g = lambda t: [P1[0]+v[0]*t,P1[1]+v[1]*t,P1[2]+v[2]*t]
    return g

# Straight line through two points

r1=para_line(punto1,punto2)
r2=para_line(punto1,punto4)
r3=para_line(punto2,punto3)
r4=para_line(punto4,punto3)
r41=para_line(punto4,punto31)

#100 points on a straight line

t_points = np.arange(0, 1, 0.01) # Creates an iterable list from 0 to 1.

l1=list(map(r1,t_points))
l2=list(map(r2,t_points))
l3=list(map(r3,t_points))
l4=list(map(r4,t_points))
l41=list(map(r41,t_points))

xl1=[x[0] for x in l1]
yl1=[x[1] for x in l1]
zl1=[x[2] for x in l1]

xl2=[x[0] for x in l2]
yl2=[x[1] for x in l2]
zl2=[x[2] for x in l2]

xl3=[x[0] for x in l3]
yl3=[x[1] for x in l3]
zl3=[x[2] for x in l3]

xl4=[x[0] for x in l4]
yl4=[x[1] for x in l4]
zl4=[x[2] for x in l4]

xl41=[x[0] for x in l41]
yl41=[x[1] for x in l41]
zl41=[x[2] for x in l41]


#into dataframes to comput latitudde and longitude

df1=pd.DataFrame(l1,columns = ['x','y','z'])
df2=pd.DataFrame(l2,columns = ['x','y','z'])
df3=pd.DataFrame(l3,columns = ['x','y','z'])
df4=pd.DataFrame(l4,columns = ['x','y','z'])

latlon1=[utm.to_latlon(xl1[i],yl1[i], 30, 'T') for i in range(len(xl1)) ]
c_lat1=[x[0] for x in latlon1]
c_lon1=[x[1] for x in latlon1]

latlon2=[utm.to_latlon(xl2[i],yl2[i], 30, 'T') for i in range(len(xl2)) ]
c_lat2=[x[0] for x in latlon2]
c_lon2=[x[1] for x in latlon2]

latlon3=[utm.to_latlon(xl3[i],yl3[i], 30, 'T') for i in range(len(xl3)) ]
c_lat3=[x[0] for x in latlon3]
c_lon3=[x[1] for x in latlon3]

latlon4=[utm.to_latlon(xl4[i],yl4[i], 30, 'T') for i in range(len(xl4)) ]
c_lat4=[x[0] for x in latlon4]
c_lon4=[x[1] for x in latlon4]

df1['lat']=c_lat1
df1['lon']=c_lon1

df2['lat']=c_lat2
df2['lon']=c_lon2

df3['lat']=c_lat3
df3['lon']=c_lon3

df4['lat']=c_lat4
df4['lon']=c_lon4

# get elevations

copernicus = CopernicusDEM(raster_paths=[DATADIR+'eu_dem_v11_E30N10.TIF'])

r1_el=copernicus.get_elevation(df1, lat_col='lat', lon_col='lon')
r2_el=copernicus.get_elevation(df2, lat_col='lat', lon_col='lon')
r3_el=copernicus.get_elevation(df3, lat_col='lat', lon_col='lon')
r4_el=copernicus.get_elevation(df4, lat_col='lat', lon_col='lon')

r1_el.to_csv(DATADIR+'r1_el.csv',index=False)
r2_el.to_csv(DATADIR+'r2_el.csv',index=False)
r3_el.to_csv(DATADIR+'r3_el.csv',index=False)
r4_el.to_csv(DATADIR+'r4_el.csv',index=False)


# with this we can calculate the topographic profile in the vertical plane determined by point1 and point2.

elvl1= r1_el['elevation'].tolist()
elvl2= r2_el['elevation'].tolist()
elvl3= r3_el['elevation'].tolist()
elvl4= r4_el['elevation'].tolist()


elvl41=elvl4

xl21=xl2+[xl4[-1]]
yl21=xl2+[yl4[-1]]
elvl21=elvl2+[elvl4[-1]]

In [ ]:
fig=go.Figure(contacts+cabs+faults+tcont+p_dat)

#bordes

fig.add_trace(go.Scatter3d(x=xl1,
                        y=yl1,
                        z=elvl1,
                       mode ='lines',
                       name='Border',
                      legendgroup='border',
                      showlegend=True,
                      line=dict(color='black',
                      width=3)
                       ))

fig.add_trace(go.Scatter3d(x=xl2,
                        y=yl2,
                        z=elvl2,
                       mode ='lines',
                       name='sur',
                      legendgroup='border',
                      showlegend=False,
                      line=dict(color='black',
                      width=3)
                       ))

fig.add_trace(go.Scatter3d(x=xl3,
                        y=yl3,
                        z=elvl3,
                       mode ='lines',
                       name='norte',
                      legendgroup='border',
                      showlegend=False,
                      line=dict(color='black',
                      width=3)
                       ))
fig.add_trace(go.Scatter3d(x=xl4,
                        y=yl4,
                        z=elvl4,
                       mode ='lines',
                       name='este',
                      legendgroup='border',
                      showlegend=False,
                      line=dict(color='black',
                      width=3)
                       ))



# texts

trace = go.Scatter3d(
                   x=namesx, y=namesy, z=namesz,
                   text=namestxt,
                   name='Heights',
                   mode="text",
                   textfont=dict(color=["black","black"],size=13),
                   hoverinfo="skip")

fig.add_trace(trace)

trace2 = go.Scatter3d(
                   x=nnx, y=nny, z=nnz,
                   text=nntxt,
                   name='Stratigraphic levels',
                   mode="text",
                   legendgroup='cabalgamientos',
                   showlegend=False,
                   textfont=dict(color='black',size=18),
                   hoverinfo="skip")

fig.add_trace(trace2)
 
# geological symbos for faults

fig.add_trace(go.Scatter3d(x=f1[0],
                        y=f1[1],
                        z=f1[2],
                       mode ='lines',
                       name='Sections',
                      legendgroup='faults',
                      showlegend=False,
                      line=dict(color='black',
                      width=5)
                       ))
fig.add_trace(go.Scatter3d(x=f2[0],
                        y=f2[1],
                        z=f2[2],
                       mode ='lines',
                       name='Sections',
                      legendgroup='faults',
                      showlegend=False,
                      line=dict(color='black',
                      width=5)
                       ))



# figure layout

camera = dict(up=dict(x=0, y=0, z=1),
              center=dict(x=0, y=0, z=0),
               eye=dict(x=0, y=-1, z=2) )

fig.update_layout( title="3D Palomeque sheets geological map",
                  #paper_bgcolor = 'black',
                 scene = dict(
                     xaxis=dict(title='UTM_X', 
                                tickfont = dict(size = 10,color = 'black'),
                                title_font_size=10,
                                titlefont_color='black', 
                                range=[cotasxy[0],cotasxy[1]],
                                backgroundcolor='white',
                                color='black',
                                gridcolor='gray'),
                     yaxis=dict(title='UTM_Y',
                                tickfont = dict(size = 10,color = 'black'),
                                title_font_size=10,
                                titlefont_color='black', 
                                range=[cotasxy[2],cotasxy[3]],  
                                backgroundcolor='white',
                                color='black',
                                gridcolor='gray'),
                      zaxis=dict(nticks=4,
                                tickfont = dict(size = 10,color = 'black'),
                                title='Elevation', 
                                title_font_size=10,
                                titlefont_color='black', 
                                range=[100,1500],
                                backgroundcolor='white',
                                color='black', 
                                gridcolor='gray'),
                     aspectratio=dict(x=2, y=1.8, z=1)),
                     #aspectmode='data'),
                     scene_camera= camera
                 ) 
              

fig.show()

## Cross-sections

Let us now integrate the seccitions in the previous 3D figure.

To shorten computing time we have saved the data in the Bezier curves calculates in the notebook `2D geological cross-sections` to cvs file, next function `get_cvs` reads the cvs files and prepare the data to be used in the definition of the 3D figure.

In [ ]:
def get_csv(lista):
    csv,nombre,color=lista
    c1=pd.read_csv(DATADIR+csv+'.csv')
    c2=c1[['x','y','z']].to_numpy()
    cx=[x[0] for x in c2]
    cy=[x[1] for x in c2]
    cz=[x[2] for x in c2]
    return [[cx,cy,cz],nombre,color]

In [ ]:
# Crossed sections

sections=pd.read_csv(DATADIR+'sections.csv')

sections1=sections[['UTM_X','UTM_Y','elevation','point']].to_numpy()
secx=[x[0] for x in sections1]
secy=[x[1] for x in sections1]
secelv=[x[2] for x in sections1]
secp=[x[3] for x in sections1]

# split the data
secx2=[(secx[i],secx[i+1]) for i in range(len(secx)-1)]
secy2=[(secy[i],secy[i+1]) for i in range(len(secx)-1)]
secelv2=[(secelv[i],secelv[i+1]) for i in range(len(secx)-1)]


points=[[(secx[i],secy[i],secelv[i]),secp[i]] for i in range(len(secx))]

# directions

d1=(secx[1]-secx[0],secy[1]-secy[0])
d2=(secx[3]-secx[2],secy[3]-secy[2])
d3=(secx[5]-secx[4],secy[5]-secy[4])
d4=(secx[7]-secx[6],secy[7]-secy[6])
d5=(secx[9]-secx[8],secy[9]-secy[8])

#normalization

d1n=(d1[0]/np.sqrt(d1[0]**2+d1[1]**2),d1[1]/np.sqrt(d1[0]**2+d1[1]**2))
d2n=(d2[0]/np.sqrt(d2[0]**2+d2[1]**2),d2[1]/np.sqrt(d2[0]**2+d2[1]**2))
d3n=(d3[0]/np.sqrt(d3[0]**2+d3[1]**2),d3[1]/np.sqrt(d3[0]**2+d3[1]**2))
d4n=(d4[0]/np.sqrt(d4[0]**2+d4[1]**2),d4[1]/np.sqrt(d4[0]**2+d4[1]**2))
d5n=(d5[0]/np.sqrt(d5[0]**2+d5[1]**2),d5[1]/np.sqrt(d5[0]**2+d5[1]**2))


The function `plane_data` uses the plot function `Scatter3d` to produce the data needed in the 3D model

In [ ]:
def plane_data(lista,grupo):
    return [go.Scatter3d(x=x[0][0], 
                         y=x[0][1], 
                         z=x[0][2],
                         mode ='lines',
                         #name=name,
                         legendgroup=grupo,
                         showlegend=False,
                         line=dict(color=x[2],width=3),
                         marker=dict(color=x[2],size=1)
                          ) for x in lista]


The function `grid` define a grid in a vertical plane determined by two points. It uses the auxiliary function `plane_y`.

In [ ]:
# The equation of the plane determined by three points, y in terms of x and z

def plane_y(p1,p2,p3,x,z):
    P1=np.array(p1)
    P2=np.array(p2)
    P3=np.array(p3)
    v1 = P3 - P1
    v2 = P2 - P1
    cp = np.cross(v1, v2)
    a, b, c = cp
    d = np.dot(cp, P3)
    if b!=0:
        return [True,(d - a * x - c * z) / b ] 
    else:
        return [False, a*x+c*z-d]

In [ ]:
def grid(A,B,r,s):
    A1=[A[0],A[1],300]
    x0=min(A[0],B[0])
    d1=abs(A[0]-B[0])//r
    x=[x0+i*d1 for i in range(r)]
    d2=(max(A[2],B[2]))//s
    z=[300+i*d2 for i in range(s)]
    y=[[plane_y(A,A1,B,x[i],z[j])[1] for i in range(r)] for j in range(s)]
    xyz=[]
    for i in range(r):
        for j in range(s):
            xyz=xyz+[(x[i],y[j][i],z[j])]
    cx=[xyz[i][0] for i in range(len(xyz))]
    cy=[xyz[i][1] for i in range(len(xyz))]
    cz=[xyz[i][2] for i in range(len(xyz))]
    return [cx,cy,cz]


The function `fil_pol` select the point of a grid that are inside a polygon.

In [ ]:
def fil_pol(poly,grid):
    xzpoly=[[poly[0][i],poly[2][i]] for i in range(len(poly[0]))]
    X1=grid[0]
    Y1=np.array(grid[1])
    Z1=grid[2]
    xz1=[[X1[i],Z1[i]] for i in range(len(X1))]

    mask1=points_in_poly(xz1,xzpoly)

    Y1[np.where(~mask1)] = 0  # to avoid importing numpy.ma

    xyz1=[[X1[i],Y1[i],Z1[i]] for i in range(len(X1)) if Y1[i]!=0]

    xx1=[x[0] for x in xyz1]
    yy1=[x[1] for x in xyz1]
    zz1=[x[2] for x in xyz1]
    return [xx1,yy1,zz1]


### Section A-A'

In [ ]:
gA=grid(points[0][0],points[1][0],200,100) #grid de puntos en el plano 200x100

# Topography A-A'

Atopo=pd.read_csv(DATADIR+'atopo.csv')


Atopo1=Atopo[['UTM_X','UTM_Y','elevation']].to_numpy()

Atopox=[x[0] for x in Atopo1]
Atopoy=[x[1] for x in Atopo1]
Atopoelv=[x[2] for x in Atopo1]

ca=[Atopox[:-10],Atopoy[:-10],Atopoelv[:-10],'A-A$^\prime$']

lA=[['calds_A','Calvillo_ds','black'],
            ['cpds_A','cpds_A','green'],
            ['e1e2_A','e1e2_A','brown'],
            ['om_A','om_A','yellow'],
            ['peds_A','peds_A','darkviolet']
           ]


corte_A=[get_csv(x) for x in lA]

c_A=plane_data(corte_A,'A')#+[p1]

corte_A[3][0]=[corte_A[3][0][0][:76],corte_A[3][0][1][:76],corte_A[3][0][2][:76]]

corte_A[2][0]=[corte_A[2][0][0][25:71],corte_A[2][0][1][25:71],corte_A[2][0][2][25:71]]

corte_A[1][0]=[corte_A[1][0][0][:76],corte_A[1][0][1][:76],corte_A[1][0][2][:76]]

corte_A[4][0]=[corte_A[4][0][0][:78],corte_A[4][0][1][:78],corte_A[4][0][2][:78]]

# contact names

calds_A=corte_A[0][0]
cpds_A=corte_A[1][0]
e1e2_A=corte_A[2][0]
om_A=corte_A[3][0]
peds_A=corte_A[4][0]

c_A=plane_data(corte_A,'A')+[
    go.Scatter3d(x=ca[0], y=ca[1], z=ca[2],
            mode ='lines',
            name='Cross-section A',
            legendgroup='A',
            showlegend=True,
            line=dict(color='black',
                      width=6)
                        )]


O2polyA=[om_A[0][::-1]+calds_A[0][34:]+[om_A[0][-1]],
         om_A[1][::-1]+calds_A[1][34:]+[om_A[1][-1]],
         om_A[2][::-1]+calds_A[2][34:]+[300]]

M1polyA=[Atopox[:27]+calds_A[0][:34]+om_A[0],
         Atopoy[:27]+calds_A[1][:34]+om_A[1],
         Atopoelv[:27]+calds_A[2][:34]+om_A[2]]


CpolyA=[cpds_A[0]+[cpds_A[0][-1]]+calds_A[0][15:][::-1], 
        cpds_A[1]+[cpds_A[1][-1]]+calds_A[1][15:][::-1], 
        cpds_A[2]+[300]+          calds_A[2][15:][::-1]]
        
        
PpolyA=[peds_A[0]+cpds_A[0][::-1]+calds_A[0][8:15][::-1],
        peds_A[1]+cpds_A[1][::-1]+calds_A[1][8:15][::-1],
        peds_A[2]+cpds_A[2][::-1]+calds_A[2][8:15][::-1]]


E1polyA=[Atopox[26:44]+e1e2_A[0]+peds_A[0][::-1]+calds_A[0][:8][::-1],
         Atopoy[26:44]+e1e2_A[1]+peds_A[1][::-1]+calds_A[1][:8][::-1],
         Atopoelv[26:44]+e1e2_A[2]+peds_A[2][::-1]+calds_A[2][:8][::-1]]
         
         
E2polyA=[e1e2_A[0][::-1]+Atopox[46:56],
        e1e2_A[1][::-1]+Atopoy[46:56],
        e1e2_A[2][::-1]+Atopoelv[46:56]]

O2A=fil_pol(O2polyA,gA)
M1A=fil_pol(M1polyA,gA)
CA=fil_pol(CpolyA,gA)
PA=fil_pol(PpolyA,gA)
E1A=fil_pol(E1polyA,gA)
E2A=fil_pol(E2polyA,gA)

O2polyA_dat=[go.Scatter3d(x=O2polyA[0], y=O2polyA[1], z=O2polyA[2],
            mode ='lines',
            name='O2A',
            legendgroup='A',
            showlegend=False,
            line=dict(color='#C19A6B',
                      width=8)
                        )]
O2A_dat=[go.Scatter3d(x=O2A[0],y=O2A[1],z=O2A[2],
                     mode ='markers',
                     name='O2A',
                     legendgroup='A',
                     showlegend=False,
                     marker=dict(color='#C19A6B',
                     size=3))]


M1polyA_dat=[go.Scatter3d(x=M1polyA[0], y=M1polyA[1], z=M1polyA[2],
            mode ='lines',
            name='M1A',
            legendgroup='A',
            showlegend=False,
            line=dict(color='yellow',
                      width=8)
                        )]
M1A_dat=[go.Scatter3d(x=M1A[0],y=M1A[1],z=M1A[2],
                     mode ='markers',
                     name='M1A',
                     legendgroup='A',
                     showlegend=False,
                     marker=dict(color='yellow',
                     size=3))]


CpolyA_dat=[go.Scatter3d(x=CpolyA[0], y=CpolyA[1], z=CpolyA[2],
            mode ='lines',
            name='CA',
            legendgroup='A',
            showlegend=False,
            line=dict(color='#D7F4D2',
                      width=8)
                        )]
CA_dat=[go.Scatter3d(x=CA[0],y=CA[1],z=CA[2],
                     mode ='markers',
                     name='A',
                     legendgroup='A',
                     showlegend=False,
                     marker=dict(color='#D7F4D2',
                     size=3))]


PpolyA_dat=[go.Scatter3d(x=PpolyA[0], y=PpolyA[1], z=PpolyA[2],
            mode ='lines',
            name='PA',
            legendgroup='A',
            showlegend=False,
            line=dict(color='#e6d7ff',
                      width=8)
                        )]
PA_dat=[go.Scatter3d(x=PA[0],y=PA[1],z=PA[2],
                     mode ='markers',
                     name='A',
                     legendgroup='A',
                     showlegend=False,
                     marker=dict(color='#e6d7ff',
                     size=3))]


E1polyA_dat=[go.Scatter3d(x=E1polyA[0], y=E1polyA[1], z=E1polyA[2],
            mode ='lines',
            name='E1A',
            legendgroup='A',
            showlegend=False,
            line=dict(color='#E97451',
                      width=8)
                        )]
E1A_dat=[go.Scatter3d(x=E1A[0],y=E1A[1],z=E1A[2],
                     mode ='markers',
                     name='E1A',
                     legendgroup='A',
                     showlegend=False,
                     marker=dict(color='#E97451',
                     size=3))]


E2polyA_dat=[go.Scatter3d(x=E2polyA[0], y=E2polyA[1], z=E2polyA[2],
            mode ='lines',
            name='E2A',
            legendgroup='A',
            showlegend=False,
            line=dict(color='pink',
                      width=8)
                        )]
E2A_dat=[go.Scatter3d(x=E2A[0],y=E2A[1],z=E2A[2],
                     mode ='markers',
                     name='E2A',
                     legendgroup='A',
                     showlegend=False,
                     marker=dict(color='pink',
                     size=3))]


In [ ]:
sa_dat=O2polyA_dat+O2A_dat+M1polyA_dat+M1A_dat+CpolyA_dat+CA_dat+PpolyA_dat+PA_dat+E1polyA_dat+E1A_dat+E2polyA_dat+E2A_dat+c_A

### Section B-B'

In [ ]:
gB=grid(points[2][0],points[3][0],200,100) #grid de puntos en el plano 100x50

# Topography B-B'

Btopo=pd.read_csv(DATADIR+'Btopo.csv')


Btopo1=Btopo[['UTM_X','UTM_Y','elevation']].to_numpy()

Btopox=[x[0] for x in Btopo1]
Btopoy=[x[1] for x in Btopo1]
Btopoelv=[x[2] for x in Btopo1]

cb=[Btopox,Btopoy,Btopoelv,'B-B$^\prime$']

Btopoelv_aux=Btopoelv.copy()
Btopoelv_aux[25]=767
Btopoelv_aux[26]=804

# list of contacts

lB=[['cal_B','cal_B','black'],
    ['cp1_B','cp1_B','green'],
    ['cp2_B','cp2_B','green'],
    ['cp3_B','cp3_B','green'],
    ['cp4_B','cp4_B','green'],
    ['e1e21_B','e1e21_B','brown'],
    ['e1e22_B','e1e22_B','brown'],
    ['e1e23_B','e1e23_B','brown'],
    ['om_B','om_B','yellow'],
    ['pal_B','pal_B','black'],
    ['pal_ds_B','pal_ds_B','black'],
    ['pe1_B','pe1_B','darkviolet'],
    ['pe2_B','pe2_B','darkviolet'],
    ['pe3_B','pe3_B','darkviolet'],
    ['pe4_B','pe4_B','darkviolet'],
    ['falla_B','falla_B','black']]

corte_B=[get_csv(x) for x in lB]

# we changed the fault depth to 300 meters

corte_B[-1][0][-1][-1]=300

# contact names

cal_B=corte_B[0][0]
cp1_B=corte_B[1][0]
cp2_B=corte_B[2][0]
cp3_B=corte_B[3][0]
cp4_B=corte_B[4][0]
e1e21_B=corte_B[5][0]
e1e22_B=corte_B[6][0]
e1e23_B=corte_B[7][0]
om_B=corte_B[8][0]
pal_B=corte_B[9][0]
pal_ds_B=corte_B[10][0]
pe1_B=corte_B[11][0]
pe2_B=corte_B[12][0]
pe3_B=corte_B[13][0]
pe4_B=corte_B[14][0]
falla_B=corte_B[15][0]

corte_B[4][0]=[corte_B[4][0][0][:60],corte_B[4][0][1][:60],corte_B[4][0][2][:60]]

corte_B[7][0]=[corte_B[7][0][0][:60],corte_B[7][0][1][:60],corte_B[7][0][2][:60]]

c_B=plane_data(corte_B,'B')+[
    go.Scatter3d(x=cb[0], y=cb[1], z=cb[2],
            mode ='lines',
            name='Cross-section B',
            legendgroup='B',
            showlegend=True,
            line=dict(color='black',
                      width=6)
                        )]

O2poly=[om_B[0][::-1]+cal_B[0][44:]+[om_B[0][-1],om_B[0][-1]],
       om_B[1][::-1]+cal_B[1][44:]+[om_B[1][-1],om_B[1][-1]],
       om_B[2][::-1]+cal_B[2][44:]+[300,om_B[2][-1]]]
O2=fil_pol(O2poly,gB)

O2pol_dat=[go.Scatter3d(x=O2poly[0], y=O2poly[1], z=O2poly[2],
            mode ='lines',
            name='O2',
            legendgroup='B',
            showlegend=False,
            line=dict(color='#C19A6B',
                      width=6)
                        )]


O2_dat=[go.Scatter3d(x=O2[0],y=O2[1],z=O2[2],
                     mode ='markers',
                     name='O2',
                     legendgroup='B',
                     showlegend=False,
                     marker=dict(color='#C19A6B',
                     size=3))]
                        

M1poly=[Btopox[:26]+cal_B[0][:44]+om_B[0],
        Btopoy[:26]+cal_B[1][:44]+om_B[1],
        Btopoelv[:26]+cal_B[2][:44]+om_B[2]] 

M1=fil_pol(M1poly,gB)

M1pol_dat=[go.Scatter3d(x=M1poly[0], y=M1poly[1], z=M1poly[2],
            mode ='lines',
            name='M1',
            legendgroup='B',
            showlegend=False,
            line=dict(color='yellow',
                      width=8)
                        )]


M1_dat=[go.Scatter3d(x=M1[0],y=M1[1],z=M1[2],
                     mode ='markers',
                     name='M1',
                     legendgroup='B',
                     showlegend=False,
                     marker=dict(color='yellow',
                     size=3))]
                        

C1poly=[cp1_B[0]+pal_B[0][60:]+cal_B[0][20:][::-1],
       cp1_B[1]+pal_B[1][60:]+cal_B[1][20:][::-1],
       cp1_B[2]+pal_B[2][60:]+cal_B[2][20:][::-1]]
C1=fil_pol(C1poly,gB)

C1pol_dat=[go.Scatter3d(x=C1poly[0], y=C1poly[1], z=C1poly[2],
            mode ='lines',
            name='C1',
            legendgroup='B',
            showlegend=False,
            line=dict(color='#D7F4D2',
                      width=8)
                        )]


C1_dat=[go.Scatter3d(x=C1[0],y=C1[1],z=C1[2],
                     mode ='markers',
                     name='C1',
                     legendgroup='B',
                     showlegend=False,
                     marker=dict(color='#D7F4D2',
                     size=3))]
                        

C2poly=[cp2_B[0]+pal_B[0][::-1],
         cp2_B[1]+pal_B[1][::-1],
         cp2_B[2]+pal_B[2][::-1]]
C3poly=[cp3_B[0]+pal_ds_B[0][47:]+[cp3_B[0][0]],
        cp3_B[1]+pal_ds_B[1][47:]+[cp3_B[1][0]],
        cp3_B[2]+pal_ds_B[2][47:]+[300]]   
C4poly=[cp4_B[0][:60]+[cp4_B[0][59]]+pal_ds_B[0][40:][::-1], 
         cp4_B[1][:60]+[cp4_B[1][59]]+pal_ds_B[1][40:][::-1], 
         cp4_B[2][:60]+[300]+pal_ds_B[2][40:][::-1]] 
C2=fil_pol(C2poly,gB)
C3=fil_pol(C3poly,gB)
C4=fil_pol(C4poly,gB)

C2pol_dat=[go.Scatter3d(x=C2poly[0], y=C2poly[1], z=C2poly[2],
            mode ='lines',
            name='C2',
            legendgroup='B',
            showlegend=False,
            line=dict(color='#D7F4D2',
                      width=8)
                        )]


C2_dat=[go.Scatter3d(x=C2[0],y=C2[1],z=C2[2],
                     mode ='markers',
                     name='C2',
                     legendgroup='B',
                     showlegend=False,
                     marker=dict(color='#D7F4D2',
                     size=3))]
                        
C3pol_dat=[go.Scatter3d(x=C3poly[0], y=C3poly[1], z=C3poly[2],
            mode ='lines',
            name='C3',
            legendgroup='B',
            showlegend=False,
            line=dict(color='#D7F4D2',
                      width=8)
                        )]


C3_dat=[go.Scatter3d(x=C3[0],y=C3[1],z=C3[2],
                     mode ='markers',
                     name='C3',
                     legendgroup='B',
                     showlegend=False,
                     marker=dict(color='#D7F4D2',
                     size=3))]
C4pol_dat=[go.Scatter3d(x=C4poly[0], y=C4poly[1], z=C4poly[2],
            mode ='lines',
            name='C4',
            legendgroup='B',
            showlegend=False,
            line=dict(color='#D7F4D2',
                      width=8)
                        )]


C4_dat=[go.Scatter3d(x=C4[0],y=C4[1],z=C4[2],
                     mode ='markers',
                     name='C4',
                     legendgroup='B',
                     showlegend=False,
                     marker=dict(color='#D7F4D2',
                     size=3))]
                        
P1poly=[pe1_B[0]+pal_B[0][30:63]+cp1_B[0][::-1]+cal_B[0][0:10][::-1]+Btopox[25:27],
        pe1_B[1]+pal_B[1][30:63]+cp1_B[1][::-1]+cal_B[1][0:10][::-1]+Btopoy[25:27],
        pe1_B[2]+pal_B[2][30:63]+cp1_B[2][::-1]+cal_B[2][0:10][::-1]+Btopoelv[25:27],
       ]
P1=fil_pol(P1poly,gB)

P1pol_dat=[go.Scatter3d(x=P1poly[0], y=P1poly[1], z=P1poly[2],
            mode ='lines',
            name='P',
            legendgroup='B',
            showlegend=False,
            line=dict(color='#e6d7ff',
                      width=8)
                        )]


P1_dat=[go.Scatter3d(x=P1[0],y=P1[1],z=P1[2],
                     mode ='markers',
                     name='P',
                     legendgroup='B',
                     showlegend=False,
                     marker=dict(color='#e6d7ff',
                     size=3))]

Btopoelv[45]=708

P2poly=[Btopox[43:46]+pe2_B[0][4:]+cp2_B[0][::-1]+pal_B[0][0:10][::-1]+[Btopox[43]],
        Btopoy[43:46]+pe2_B[1][4:]+cp2_B[1][::-1]+pal_B[1][0:10][::-1]+[Btopoy[43]],
        Btopoelv[43:46]+pe2_B[2][4:]+cp2_B[2][::-1]+pal_B[2][0:10][::-1]+[Btopoelv[43]]
        ]
P2=fil_pol(P2poly,gB)

P2pol_dat=[go.Scatter3d(x=P2poly[0], y=P2poly[1], z=P2poly[2],
            mode ='lines',
            name='P',
            legendgroup='B',
            showlegend=False,
            line=dict(color='#e6d7ff',
                      width=8)
                        )]


P2_dat=[go.Scatter3d(x=P2[0],y=P2[1],z=P2[2],
                     mode ='markers',
                     name='P2',
                     legendgroup='B',
                     showlegend=False,
                     marker=dict(color='#e6d7ff',
                     size=3))]
                        

P3poly=[pe3_B[0]+pal_ds_B[0][25:30]+cp3_B[0][::-1]+[pe3_B[0][0]],
         pe3_B[1]+pal_ds_B[1][25:30]+cp3_B[1][::-1]+[pe3_B[1][0]],
         pe3_B[2]+pal_ds_B[2][25:30]+cp3_B[2][::-1]+[pe3_B[2][0]]] 
P3=fil_pol(P3poly,gB)

P3pol_dat=[go.Scatter3d(x=P3poly[0], y=P3poly[1], z=P3poly[2],
            mode ='lines',
            name='P3',
            legendgroup='B',
            showlegend=False,
            line=dict(color='#e6d7ff',
                      width=8)
                        )]


P3_dat=[go.Scatter3d(x=P3[0],y=P3[1],z=P3[2],
                     mode ='markers',
                     name='P3',
                     legendgroup='B',
                     showlegend=False,
                     marker=dict(color='#e6d7ff',
                     size=3))]

P4poly=[cp4_B[0][:60][::-1]+pal_ds_B[0][20:28]+pe4_B[0]+[cp4_B[0][60]],
        cp4_B[1][:60][::-1]+pal_ds_B[1][20:28]+pe4_B[1]+[cp4_B[1][60]],
        cp4_B[2][:60][::-1]+pal_ds_B[2][20:28]+pe4_B[2]+[cp4_B[2][60]]]
P4=fil_pol(P4poly,gB)

P4pol_dat=[go.Scatter3d(x=P4poly[0], y=P4poly[1], z=P4poly[2],
            mode ='lines',
            name='P4',
            legendgroup='B',
            showlegend=False,
            line=dict(color='#e6d7ff',
                      width=8)
                        )]


P4_dat=[go.Scatter3d(x=P4[0],y=P4[1],z=P4[2],
                     mode ='markers',
                     name='P4',
                     legendgroup='B',
                     showlegend=False,
                     marker=dict(color='#e6d7ff',
                     size=3))]
                        

E11poly=[pe1_B[0][::-1]+Btopox[27:36]+e1e21_B[0]+pal_B[0][11:30],
         pe1_B[1][::-1]+Btopoy[27:36]+e1e21_B[1]+pal_B[1][11:30],
         pe1_B[2][::-1]+Btopoelv[27:36]+e1e21_B[2]+pal_B[2][11:30]]
                    

E12poly=[pe2_B[0][::-1]+Btopox[45:56],
         pe2_B[1][::-1]+Btopoy[45:56],
         pe2_B[2][::-1]+Btopoelv[45:56]]
                    
E13poly=[e1e22_B[0]+pal_ds_B[0][7:20]+pe3_B[0][::-1],
         e1e22_B[1]+pal_ds_B[1][7:20]+pe3_B[1][::-1],
         e1e22_B[2]+pal_ds_B[2][7:20]+pe3_B[2][::-1]]
                    
E14poly=[pe4_B[0][::-1]+pal_ds_B[0][:20][::-1]+Btopox[60:70]+e1e23_B[0][10:65],
         pe4_B[1][::-1]+pal_ds_B[1][:20][::-1]+Btopoy[60:70]+e1e23_B[1][10:65],
         pe4_B[2][::-1]+pal_ds_B[2][:20][::-1]+Btopoelv[60:70]+e1e23_B[2][10:65]]
E11=fil_pol(E11poly,gB)
E12=fil_pol(E12poly,gB)
E13=fil_pol(E13poly,gB)
E14=fil_pol(E14poly,gB)

E11pol_dat=[go.Scatter3d(x=E11poly[0], y=E11poly[1], z=E11poly[2],
            mode ='lines',
            name='E11',
            legendgroup='B',
            showlegend=False,
            line=dict(color='#E97451',
                      width=8)
                        )]


E11_dat=[go.Scatter3d(x=E11[0],y=E11[1],z=E11[2],
                     mode ='markers',
                     name='E11',
                     legendgroup='B',
                     showlegend=False,
                     marker=dict(color='#E97451',
                     size=3))]
E12pol_dat=[go.Scatter3d(x=E12poly[0], y=E12poly[1], z=E12poly[2],
            mode ='lines',
            name='E12',
            legendgroup='B',
            showlegend=False,
            line=dict(color='#E97451',
                      width=8)
                        )]


E12_dat=[go.Scatter3d(x=E12[0],y=E12[1],z=E12[2],
                     mode ='markers',
                     name='E12',
                     legendgroup='B',
                     showlegend=False,
                     marker=dict(color='#E97451',
                     size=3))]
E13pol_dat=[go.Scatter3d(x=E13poly[0], y=E13poly[1], z=E13poly[2],
            mode ='lines',
            name='E13',
            legendgroup='B',
            showlegend=False,
            line=dict(color='#E97451',
                      width=8)
                        )]


E13_dat=[go.Scatter3d(x=E13[0],y=E13[1],z=E13[2],
                     mode ='markers',
                     name='E13',
                     legendgroup='B',
                     showlegend=False,
                     marker=dict(color='#E97451',
                     size=3))]
E14pol_dat=[go.Scatter3d(x=E14poly[0], y=E14poly[1], z=E14poly[2],
            mode ='lines',
            name='E14',
            legendgroup='B',
            showlegend=False,
            line=dict(color='#E97451',
                      width=8)
                        )]


E14_dat=[go.Scatter3d(x=E14[0],y=E14[1],z=E14[2],
                     mode ='markers',
                     name='E14',
                     legendgroup='B',
                     showlegend=False,
                     marker=dict(color='#E97451',
                     size=3))]
                                                
                                            

E21poly=[e1e21_B[0][::-1]+Btopox[37:40]+pal_B[0][:11],
         e1e21_B[1][::-1]+Btopoy[37:40]+pal_B[1][:11],
         e1e21_B[2][::-1]+Btopoelv[37:40]+pal_B[2][:11]]
                    
E22poly=[Btopox[55:58]+pal_ds_B[0][:7]+e1e22_B[0][::-1],
         Btopoy[55:58]+pal_ds_B[1][:7]+e1e22_B[1][::-1],
         Btopoelv[55:58]+pal_ds_B[2][:7]+e1e22_B[2][::-1]]
                     
E23poly=[e1e23_B[0][:60][::-1]+Btopox[70:],
         e1e23_B[1][:60][::-1]+Btopoy[70:],
         e1e23_B[2][:60][::-1]+Btopoelv[70:]]

E21=fil_pol(E21poly,gB)
E22=fil_pol(E22poly,gB)
E23=fil_pol(E23poly,gB)

E21pol_dat=[go.Scatter3d(x=E21poly[0], y=E21poly[1], z=E21poly[2],
            mode ='lines',
            name='E21',
            legendgroup='B',
            showlegend=False,
            line=dict(color='pink',
                      width=8)
                        )]


E21_dat=[go.Scatter3d(x=E21[0],y=E21[1],z=E21[2],
                     mode ='markers',
                     name='E21',
                     legendgroup='B',
                     showlegend=False,
                     marker=dict(color='pink',
                     size=3))]
E22pol_dat=[go.Scatter3d(x=E22poly[0], y=E22poly[1], z=E22poly[2],
            mode ='lines',
            name='E22',
            legendgroup='B',
            showlegend=False,
            line=dict(color='pink',
                      width=8)
                        )]


E22_dat=[go.Scatter3d(x=E22[0],y=E22[1],z=E22[2],
                     mode ='markers',
                     name='E22',
                     legendgroup='B',
                     showlegend=False,
                     marker=dict(color='pink',
                     size=3))]
E23pol_dat=[go.Scatter3d(x=E23poly[0], y=E23poly[1], z=E23poly[2],
            mode ='lines',
            name='E23',
            legendgroup='B',
            showlegend=False,
            line=dict(color='pink',
                      width=8)
                        )]


E23_dat=[go.Scatter3d(x=E23[0],y=E23[1],z=E23[2],
                     mode ='markers',
                     name='E23',
                     legendgroup='B',
                     showlegend=False,
                     marker=dict(color='pink',
                     size=3))]                           

In [ ]:
sb_dat=O2pol_dat+O2_dat+M1pol_dat+M1_dat+C1pol_dat+C1_dat+C2pol_dat+C2_dat+C3pol_dat+C3_dat+C4pol_dat+C4_dat+P1pol_dat+P1_dat+P2pol_dat+P2_dat+P3pol_dat+P3_dat+P4pol_dat+P4_dat+E11pol_dat+E11_dat+E12pol_dat+E12_dat+E13pol_dat+E13_dat+E14pol_dat+E14_dat+E21pol_dat+E21_dat+E22pol_dat+E22_dat+E23pol_dat+E23_dat+c_B

### Section C-C'

In [ ]:
gC=grid(points[4][0],points[5][0],200,100) #grid de puntos en el plano 100x50

# Topography C-C'

Ctopo=pd.read_csv(DATADIR+'Ctopo.csv')


Ctopo1=Ctopo[['UTM_X','UTM_Y','elevation']].to_numpy()

Ctopox=[x[0] for x in Ctopo1]
Ctopoy=[x[1] for x in Ctopo1]
Ctopoelv=[x[2] for x in Ctopo1]

cc=[Ctopox[:-6],Ctopoy[:-6],Ctopoelv[:-6],'C-C$^\prime$']

lC=[['cal_C','cal_C','black'],
    ['cp1_C','cp1_C','green'],
    ['cp2_C','cp2_C','green'],
    ['e1e21_C','e1e21_C','brown'],['e1e22_C','e1e22_C','brown'],
    ['e2e3_C','e2e3_C','pink'],
    ['om_C','om_C','yellow'],
    ['pal_C','pal_C','black'],
    ['pe1_C','pe1_C','darkviolet'],['pe2_C','pe2_C','darkviolet']]

corte_C=[get_csv(x) for x in lC]

corte_C[4][0]=[corte_C[4][0][0][8:68],corte_C[4][0][1][8:68],corte_C[4][0][2][8:68]]
corte_C[5][0]=[corte_C[5][0][0][:57],corte_C[5][0][1][:57],corte_C[5][0][2][:57]]
corte_C[9][0]=[corte_C[9][0][0][:65],corte_C[9][0][1][:65],corte_C[9][0][2][:65]]

# contacts names

cal_C=corte_C[0][0]
cp1_C=corte_C[1][0]
cp2_C=corte_C[2][0]

e1e21_C=corte_C[3][0]
e1e22_C=corte_C[4][0]

e2e3_C=corte_C[5][0]

om_C=corte_C[6][0]
pal_C=corte_C[7][0]

pe1_C=corte_C[8][0]
pe2_C=corte_C[9][0]

c_C=plane_data(corte_C,'C')+[
    go.Scatter3d(x=cc[0], y=cc[1], z=cc[2],
            mode ='lines',
            name='Cross-section C',
            legendgroup='C',#'C',
            showlegend=True,
            line=dict(color='black',
                      width=6)
                        )]

pCO=[om_C[0][::-1]+cal_C[0][27:]+[om_C[0][-1]],
    om_C[1][::-1]+cal_C[1][27:]+[om_C[1][-1]],
    om_C[2][::-1]+cal_C[2][27:]+[300]]


pCM=[Ctopox[:20]+cal_C[0][:27]+om_C[0],
    Ctopoy[:20]+cal_C[1][:27]+om_C[1],
    Ctopoelv[:20]+cal_C[2][:27]+om_C[2]]


pCC1=[cal_C[0][20:][::-1]+cp1_C[0]+pal_C[0][43:],
      cal_C[1][20:][::-1]+cp1_C[1]+pal_C[1][43:],
      cal_C[2][20:][::-1]+cp1_C[2]+pal_C[2][43:]]

pCC2=[pal_C[0][10:][::-1]+cp2_C[0],
      pal_C[1][10:][::-1]+cp2_C[1],
      pal_C[2][10:][::-1]+cp2_C[2]]
                   

pCP1=[cp1_C[0][::-1]+cal_C[0][:10][::-1]+Ctopox[20:23]+pe1_C[0]+pal_C[0][20:42],
     cp1_C[1][::-1]+cal_C[1][:10][::-1]+Ctopoy[20:23]+pe1_C[1]+pal_C[1][20:42],
     cp1_C[2][::-1]+cal_C[2][:10][::-1]+Ctopoelv[20:23]+pe1_C[2]+pal_C[2][20:42]]

pCP2=[cp2_C[0][::-1]+pal_C[0][:10][::-1]+Ctopox[40:42]+pe2_C[0]+[pe2_C[0][-1]],
     cp2_C[1][::-1]+pal_C[1][:10][::-1]+Ctopoy[40:42]+pe2_C[1]+[pe2_C[1][-1]],
     cp2_C[2][::-1]+pal_C[2][:10][::-1]+Ctopoelv[40:42]+pe2_C[2]+[300]]

pCE11=[pe1_C[0][::-1]+Ctopox[25:32]+e1e21_C[0]+pal_C[0][10:25],
      pe1_C[1][::-1]+Ctopoy[25:32]+e1e21_C[1]+pal_C[1][10:25],
      pe1_C[2][::-1]+Ctopoelv[25:32]+e1e21_C[2]+pal_C[2][10:25]]


pCE12=[pe2_C[0][::-1]+Ctopox[43:50]+e1e22_C[0],
      pe2_C[1][::-1]+Ctopoy[43:50]+e1e22_C[1],
      pe2_C[2][::-1]+Ctopoelv[43:50]+e1e22_C[2]]


pCE21=[e1e21_C[0][::-1]+Ctopox[37:39]+pal_C[0][:10],
      e1e21_C[1][::-1]+Ctopoy[37:39]+pal_C[1][:10],
      e1e21_C[2][::-1]+Ctopoelv[37:39]+pal_C[2][:10]]

pCE22=[e1e22_C[0][::-1]+Ctopox[50:55]+e2e3_C[0],
       e1e22_C[1][::-1]+Ctopoy[50:55]+e2e3_C[1],
       e1e22_C[2][::-1]+Ctopoelv[50:55]+e2e3_C[2]]


pCE3=[e2e3_C[0][::-1]+Ctopox[60:-6],
     e2e3_C[1][::-1]+Ctopoy[60:-6],
     e2e3_C[2][::-1]+Ctopoelv[60:-6]]

CO=fil_pol(pCO,gC)

CM=fil_pol(pCM,gC)
CC1=fil_pol(pCC1,gC)
CC2=fil_pol(pCC2,gC)
CP1=fil_pol(pCP1,gC)
CP2=fil_pol(pCP2,gC)
CE11=fil_pol(pCE11,gC)
CE12=fil_pol(pCE12,gC)
CE21=fil_pol(pCE21,gC)
CE22=fil_pol(pCE22,gC)
CE3=fil_pol(pCE3,gC)

pCO_dat=[go.Scatter3d(x=pCO[0], y=pCO[1], z=pCO[2],
            mode ='lines',
            name='O2C',
            legendgroup='C',#'O2C',
            showlegend=False,
            line=dict(color='#C19A6B',
                      width=8)
                        )]
CO_dat=[go.Scatter3d(x=CO[0],y=CO[1],z=CO[2],
                     mode ='markers',
                     name='O2C',
                     legendgroup='C',#'O2C',
                     showlegend=False,
                     marker=dict(color='#C19A6B',
                     size=3))]


pCM_dat=[go.Scatter3d(x=pCM[0], y=pCM[1], z=pCM[2],
            mode ='lines',
            name='M2C',
            legendgroup='C',#'M2C',
            showlegend=False,
            line=dict(color='yellow',
                      width=8)
                        )]
CM_dat=[go.Scatter3d(x=CM[0],y=CM[1],z=CM[2],
                     mode ='markers',
                     name='M2C',
                     legendgroup='C',#'M2C',
                     showlegend=False,
                     marker=dict(color='yellow',
                     size=3))]
pCC1_dat=[go.Scatter3d(x=pCC1[0], y=pCC1[1], z=pCC1[2],
            mode ='lines',
            name='C12C',
            legendgroup='C',#'C12C',
            showlegend=False,
            line=dict(color='#D7F4D2',
                      width=8)
                        )]
CC1_dat=[go.Scatter3d(x=CC1[0],y=CC1[1],z=CC1[2],
                     mode ='markers',
                     name='C12C',
                     legendgroup='C',#'C12C',
                     showlegend=False,
                     marker=dict(color='#D7F4D2',
                     size=3))]


pCC2_dat=[go.Scatter3d(x=pCC2[0], y=pCC2[1], z=pCC2[2],
            mode ='lines',
            name='C22C',
            legendgroup='C',#'C22C',
            showlegend=False,
            line=dict(color='#D7F4D2',
                      width=8)
                        )]
CC2_dat=[go.Scatter3d(x=CC2[0],y=CC2[1],z=CC2[2],
                     mode ='markers',
                     name='C22C',
                     legendgroup='C',#'C22C',
                     showlegend=False,
                     marker=dict(color='#D7F4D2',
                     size=3))]
pCP1_dat=[go.Scatter3d(x=pCP1[0], y=pCP1[1], z=pCP1[2],
            mode ='lines',
            name='P1C',
            legendgroup='C',#'P1C',
            showlegend=False,
            line=dict(color='#e6d7ff',
                      width=8)
                        )]
CP1_dat=[go.Scatter3d(x=CP1[0],y=CP1[1],z=CP1[2],
                     mode ='markers',
                     name='P1C',
                     legendgroup='C',#'P1C',
                     showlegend=False,
                     marker=dict(color='#e6d7ff',
                     size=3))]


pCP2_dat=[go.Scatter3d(x=pCP2[0], y=pCP2[1], z=pCP2[2],
            mode ='lines',
            name='P2C',
            legendgroup='C',#'P2C',
            showlegend=False,
            line=dict(color='#e6d7ff',
                      width=8)
                        )]
CP2_dat=[go.Scatter3d(x=CP2[0],y=CP2[1],z=CP2[2],
                     mode ='markers',
                     name='P2C',
                     legendgroup='C',#'P2C',
                     showlegend=False,
                     marker=dict(color='#e6d7ff',
                     size=3))]
pCE11_dat=[go.Scatter3d(x=pCE11[0], y=pCE11[1], z=pCE11[2],
            mode ='lines',
            name='E11C',
            legendgroup='C',#'E11C',
            showlegend=False,
            line=dict(color='#E97451',
                      width=8)
                        )]
CE11_dat=[go.Scatter3d(x=CE11[0],y=CE11[1],z=CE11[2],
                     mode ='markers',
                     name='E11C',
                     legendgroup='C',#'E11C',
                     showlegend=False,
                     marker=dict(color='#E97451',
                     size=3))]


pCE12_dat=[go.Scatter3d(x=pCE12[0], y=pCE12[1], z=pCE12[2],
            mode ='lines',
            name='E12C',
            legendgroup='C',#'E12C',
            showlegend=False,
            line=dict(color='#E97451',
                      width=8)
                        )]
CE12_dat=[go.Scatter3d(x=CE12[0],y=CE12[1],z=CE12[2],
                     mode ='markers',
                     name='E12C',
                     legendgroup='C',#'E12C',
                     showlegend=False,
                     marker=dict(color='#E97451',
                     size=3))]


pCE21_dat=[go.Scatter3d(x=pCE21[0], y=pCE21[1], z=pCE21[2],
            mode ='lines',
            name='E21C',
            legendgroup='C',#'E21C',
            showlegend=False,
            line=dict(color='pink',
                      width=8)
                        )]
CE21_dat=[go.Scatter3d(x=CE21[0],y=CE21[1],z=CE21[2],
                     mode ='markers',
                     name='E21C',
                     legendgroup='C',#'E21C',
                     showlegend=False,
                     marker=dict(color='pink',
                     size=3))]
pCE22_dat=[go.Scatter3d(x=pCE22[0], y=pCE22[1], z=pCE22[2],
            mode ='lines',
            name='E22C',
            legendgroup='C',#'E22C',
            showlegend=False,
            line=dict(color='pink',
                      width=8)
                        )]
CE22_dat=[go.Scatter3d(x=CE22[0],y=CE22[1],z=CE22[2],
                     mode ='markers',
                     name='E22C',
                     legendgroup='C',#'E22C',
                     showlegend=False,
                     marker=dict(color='pink',
                     size=3))]


pCE3_dat=[go.Scatter3d(x=pCE3[0], y=pCE3[1], z=pCE3[2],
            mode ='lines',
            name='E3C',
            legendgroup='C',#'E3C',
            showlegend=False,
            line=dict(color='#D1BEA8',
                      width=8)
                        )]
CE3_dat=[go.Scatter3d(x=CE3[0],y=CE3[1],z=CE3[2],
                     mode ='markers',
                     name='E3C',
                     legendgroup='C',#'E3C',
                     showlegend=False,
                     marker=dict(color='#D1BEA8',
                     size=3))]




In [ ]:
sc_dat=pCO_dat+CO_dat+pCM_dat+CM_dat+pCC1_dat+CC1_dat+pCC2_dat+CC2_dat+pCP1_dat+CP1_dat+pCP2_dat+CP2_dat+pCE11_dat+CE11_dat+pCE12_dat+CE12_dat+pCE21_dat+CE21_dat+pCE22_dat+CE22_dat+pCE3_dat+CE3_dat+c_C

### Section D-D'

In [ ]:
def gridd(A,B,r,s):
    A1=[A[0],A[1],300]
    x0=min(A[0],B[0])
    d1=abs(A[0]-B[0])//r
    x=[x0+i*d1 for i in range(17,r+30,1)]
    d2=(max(A[2],B[2]))//s
    z=[300+i*d2 for i in range(s)]
    y=[[plane_y(A,A1,B,x[i],z[j])[1] for i in range(len(x))] for j in range(s)]
    xyz=[]
    for i in range(len(x)):
        for j in range(s):
            xyz=xyz+[(x[i],y[j][i],z[j])]
    cx=[xyz[i][0] for i in range(len(xyz))]
    cy=[xyz[i][1] for i in range(len(xyz))]
    cz=[xyz[i][2] for i in range(len(xyz))]
    return [cx,cy,cz]

PP=(619785.1419615997, 4197401.308092775, 700)
QQ=(620960.668645401, 4195069.523849677, 700)

gD=gridd(points[6][0],points[7][0],200,100) #grid de puntos en el plano 100x50

# Topography D-D'

Dtopo=pd.read_csv(DATADIR+'Dtopo.csv')


Dtopo1=Dtopo[['UTM_X','UTM_Y','elevation']].to_numpy()

Dtopox=[x[0] for x in Dtopo1]
Dtopoy=[x[1] for x in Dtopo1]
Dtopoelv=[x[2] for x in Dtopo1]

Dtopox=Dtopox[5:]
Dtopoy=Dtopoy[5:]
Dtopoelv=Dtopoelv[5:]
cd=[Dtopox,Dtopoy,Dtopoelv,'D-D$^\prime$']

lD=[['cal_D','cal_D','black'],
    ['pal_D','pal_D','black'],
    ['om_D','om_D','yellow'],
    ['cp1_D','cp1_D','green'],
    ['pe0_D','pe0_D','darkviolet'],
    ['pe1_D','pe1_D','darkviolet'],
    ['e1e20_D','e1e20_D','brown'],
    ['e1e21_D','e1e21_D','brown'],
    ['e2e3_D','e2e3_D','pink'],
    ['e3e0_D','e3e0_D','#D19A6B'],
    ]
    
corte_D=[get_csv(x) for x in lD]

corte_D[0][0]=[corte_D[0][0][0][:96],corte_D[0][0][1][:96],corte_D[0][0][2][:96]]
corte_D[5][0]=[corte_D[5][0][0][:96],corte_D[5][0][1][:96],corte_D[5][0][2][:96]]
corte_D[7][0]=[corte_D[7][0][0][:88],corte_D[7][0][1][:88],corte_D[7][0][2][:88]]
corte_D[8][0]=[corte_D[8][0][0][:83],corte_D[8][0][1][:83],corte_D[8][0][2][:83]]
corte_D[9][0]=[corte_D[9][0][0][10:64],corte_D[9][0][1][10:64],corte_D[9][0][2][10:64]]
c_D=plane_data(corte_D,'D')+[go.Scatter3d(x=cd[0], y=cd[1], z=cd[2],
            mode ='lines',
            name='Cross-section D',
            legendgroup='D',#'D',
            showlegend=True,
            line=dict(color='black',
                      width=6)
                        )]#+[p4]

# contacts names

cal_D=corte_D[0][0]
pal_D=corte_D[1][0]
om_D=corte_D[2][0]

cp1_D=corte_D[3][0]
pe0_D=corte_D[4][0]

pe1_D=corte_D[5][0]

e1e20_D=corte_D[6][0]
e1e21_D=corte_D[7][0]

e2e3_D=corte_D[8][0]
e3e0_D=corte_D[9][0]

pDO=[om_D[0][::-1]+cal_D[0][27:]+[om_D[0][-1]],
    om_D[1][::-1]+cal_D[1][27:]+[om_D[1][-1]],
    om_D[2][::-1]+cal_D[2][27:]+[300.52553074634613]]


pDM=[Dtopox[:26]+cal_D[0][:18]+om_D[0],
    Dtopoy[:26]+cal_D[1][:18]+om_D[1],
    Dtopoelv[:26]+cal_D[2][:18]+om_D[2]]

pDC=[pal_D[0][::-1]+cp1_D[0],
     pal_D[1][::-1]+cp1_D[1],
     pal_D[2][::-1]+cp1_D[2]]
                   

pDP1=[cal_D[0][15:][::-1]+pe0_D[0]+pal_D[0][15:], 
      cal_D[1][15:][::-1]+pe0_D[1]+pal_D[1][15:],
      cal_D[2][15:][::-1]+pe0_D[2]+pal_D[2][15:]]

pDP2=[cp1_D[0][::-1]+Dtopox[29:32]+pe1_D[0]+[pe1_D[0][-1]],
     cp1_D[1][::-1]+Dtopoy[29:32]+pe1_D[1]+[pe1_D[1][-1]],
     cp1_D[2][::-1]+Dtopoelv[29:32]+pe1_D[2]+[300]]

pDE11=[pe0_D[0][::-1]+cal_D[0][7:15][::-1]+e1e20_D[0]+pal_D[0][7:18],
      pe0_D[1][::-1]+cal_D[1][7:15][::-1]+e1e20_D[1]+pal_D[1][7:18],
      pe0_D[2][::-1]+cal_D[2][7:15][::-1]+e1e20_D[2]+pal_D[2][7:18]]

pDE12=[pe1_D[0][::-1]+Dtopox[33:43]+e1e21_D[0],
      pe1_D[1][::-1]+Dtopoy[33:43]+e1e21_D[1],
      pe1_D[2][::-1]+Dtopoelv[33:43]+e1e21_D[2]]
pDE21=[e1e20_D[0][::-1]+cal_D[0][:4][::-1]+Dtopox[27:29]+pal_D[0][:7],
     e1e20_D[1][::-1]+cal_D[1][:4][::-1]+Dtopoy[27:29]+pal_D[1][:7],
     e1e20_D[2][::-1]+cal_D[2][:4][::-1]+Dtopoelv[27:29]+pal_D[2][:7]]
pDE22=[e1e21_D[0][::-1]+Dtopox[44:46]+e2e3_D[0],
     e1e21_D[1][::-1]+Dtopoy[44:46]+e2e3_D[1],
     e1e21_D[2][::-1]+Dtopoelv[44:46]+e2e3_D[2]]
pDE3=[e2e3_D[0][::-1]+Dtopox[53:59]+e3e0_D[0],
    e2e3_D[1][::-1]+Dtopoy[53:59]+e3e0_D[1],
    e2e3_D[2][::-1]+Dtopoelv[53:59]+e3e0_D[2]]
pDE4=[e3e0_D[0][::-1]+Dtopox[64:],
     e3e0_D[1][::-1]+Dtopoy[64:],
     e3e0_D[2][::-1]+Dtopoelv[64:]]

DO=fil_pol(pDO,gD)
DM=fil_pol(pDM,gD)
DC=fil_pol(pDC,gD)
DP1=fil_pol(pDP1,gD)
DP2=fil_pol(pDP2,gD)
DE11=fil_pol(pDE11,gD)
DE12=fil_pol(pDE12,gD)
DE21=fil_pol(pDE21,gD)
DE22=fil_pol(pDE22,gD)
DE3=fil_pol(pDE3,gD)
DE4=fil_pol(pDE4,gD)

pDO_dat=[go.Scatter3d(x=pDO[0], y=pDO[1], z=pDO[2],
            mode ='lines',
            name='O2D',
            legendgroup='D',#'O2D',
            showlegend=False,
            line=dict(color='#D19A6B',
                      width=8)
                        )]
DO_dat=[go.Scatter3d(x=DO[0],y=DO[1],z=DO[2],
                     mode ='markers',
                     name='O2D',
                     legendgroup='D',#'O2D',
                     showlegend=False,
                     marker=dict(color='#D19A6B',
                     size=3))]


pDM_dat=[go.Scatter3d(x=pDM[0], y=pDM[1], z=pDM[2],
            mode ='lines',
            name='M2D',
            legendgroup='D',#'M2D',
            showlegend=False,
            line=dict(color='yellow',
                      width=8)
                        )]
DM_dat=[go.Scatter3d(x=DM[0],y=DM[1],z=DM[2],
                     mode ='markers',
                     name='M2D',
                     legendgroup='D',#'M2D',
                     showlegend=False,
                     marker=dict(color='yellow',
                     size=3))]


pDC_dat=[go.Scatter3d(x=pDC[0], y=pDC[1], z=pDC[2],
            mode ='lines',
            name='C2D',
            legendgroup='D',#'C2D',
            showlegend=False,
            line=dict(color='#D7F4D2',
                      width=8)
                        )]
DC_dat=[go.Scatter3d(x=DC[0],y=DC[1],z=DC[2],
                     mode ='markers',
                     name='C2D',
                     legendgroup='D',#'C2D',
                     showlegend=False,
                     marker=dict(color='#D7F4D2',
                     size=3))]
pDP1_dat=[go.Scatter3d(x=pDP1[0], y=pDP1[1], z=pDP1[2],
            mode ='lines',
            name='P1D',
            legendgroup='D',#'P1D',
            showlegend=False,
            line=dict(color='#e6d7ff',
                      width=8)
                        )]
DP1_dat=[go.Scatter3d(x=DP1[0],y=DP1[1],z=DP1[2],
                     mode ='markers',
                     name='P1D',
                     legendgroup='D',#'P1D',
                     showlegend=False,
                     marker=dict(color='#e6d7ff',
                     size=3))]


pDP2_dat=[go.Scatter3d(x=pDP2[0], y=pDP2[1], z=pDP2[2],
            mode ='lines',
            name='P2D',
            legendgroup='D',#'P2D',
            showlegend=False,
            line=dict(color='#e6d7ff',
                      width=8)
                        )]
DP2_dat=[go.Scatter3d(x=DP2[0],y=DP2[1],z=DP2[2],
                     mode ='markers',
                     name='P2D',
                     legendgroup='D',#'P2D',
                     showlegend=False,
                     marker=dict(color='#e6d7ff',
                     size=3))]


pDE11_dat=[go.Scatter3d(x=pDE11[0], y=pDE11[1], z=pDE11[2],
            mode ='lines',
            name='E11D',
            legendgroup='D',#'E11D',
            showlegend=False,
            line=dict(color='#E97451',
                      width=8)
                        )]
DE11_dat=[go.Scatter3d(x=DE11[0],y=DE11[1],z=DE11[2],
                     mode ='markers',
                     name='E11D',
                     legendgroup='D',#'E11D',
                     showlegend=False,
                     marker=dict(color='#E97451',
                     size=3))]
pDE12_dat=[go.Scatter3d(x=pDE12[0], y=pDE12[1], z=pDE12[2],
            mode ='lines',
            name='E12D',
            legendgroup='D',#'E12D',
            showlegend=False,
            line=dict(color='#E97451',
                      width=8)
                        )]
DE12_dat=[go.Scatter3d(x=DE12[0],y=DE12[1],z=DE12[2],
                     mode ='markers',
                     name='E12D',
                     legendgroup='D',#'E12D',
                     showlegend=False,
                     marker=dict(color='#E97451',
                     size=3))]


pDE21_dat=[go.Scatter3d(x=pDE21[0], y=pDE21[1], z=pDE21[2],
            mode ='lines',
            name='E21D',
            legendgroup='D',#'E21D',
            showlegend=False,
            line=dict(color='pink',
                      width=8)
                        )]
DE21_dat=[go.Scatter3d(x=DE21[0],y=DE21[1],z=DE21[2],
                     mode ='markers',
                     name='E21D',
                     legendgroup='D',#'E21D',
                     showlegend=False,
                     marker=dict(color='pink',
                     size=3))]


pDE22_dat=[go.Scatter3d(x=pDE22[0], y=pDE22[1], z=pDE22[2],
            mode ='lines',
            name='E22D',
            legendgroup='D',#'E22D',
            showlegend=False,
            line=dict(color='pink',
                      width=8)
                        )]
DE22_dat=[go.Scatter3d(x=DE22[0],y=DE22[1],z=DE22[2],
                     mode ='markers',
                     name='E22D',
                     legendgroup='D',#'E22D',
                     showlegend=False,
                     marker=dict(color='pink',
                     size=3))]


pDE3_dat=[go.Scatter3d(x=pDE3[0], y=pDE3[1], z=pDE3[2],
            mode ='lines',
            name='E3D',
            legendgroup='D',#'E3D',
            showlegend=False,
            line=dict(color='#D1BEA8',
                      width=8)
                        )]
DE3_dat=[go.Scatter3d(x=DE3[0],y=DE3[1],z=DE3[2],
                     mode ='markers',
                     name='E3D',
                     legendgroup='D',#'E3D',
                     showlegend=False,
                     marker=dict(color='#D1BEA8',
                     size=3))]
pDE4_dat=[go.Scatter3d(x=pDE4[0], y=pDE4[1], z=pDE4[2],
            mode ='lines',
            name='E4D',
            legendgroup='D',#'E4D',
            showlegend=False,
            line=dict(color='#D19A6B',
                      width=8)
                        )]
DE4_dat=[go.Scatter3d(x=DE4[0],y=DE4[1],z=DE4[2],
                     mode ='markers',
                     name='E4D',
                     legendgroup='D',#'E4D',
                     showlegend=False,
                     marker=dict(color='#D19A6B',
                     size=3))]

In [ ]:
sd_dat=pDO_dat+DO_dat+pDM_dat+DM_dat+pDC_dat+DC_dat+pDP1_dat+DP1_dat+pDP2_dat+DP2_dat+pDE11_dat+DE11_dat+pDE12_dat+DE12_dat+pDE21_dat+DE21_dat+pDE22_dat+DE22_dat+pDE3_dat+DE3_dat+pDE4_dat+DE4_dat+c_D
             

### Section E-E'

In [ ]:
gE=gridd(points[8][0],points[9][0],200,100) #grid de puntos en el plano 100x50

# Topography E-E'

Etopo=pd.read_csv(DATADIR+'Etopo.csv')


Etopo1=Etopo[['UTM_X','UTM_Y','elevation']].to_numpy()

Etopox=[x[0] for x in Etopo1]
Etopoy=[x[1] for x in Etopo1]
Etopoelv=[x[2] for x in Etopo1]
Etopot=[np.sqrt(Etopox[i]**2+Etopoy[i]**2) for i in range(len(Etopox))]

Etopox=Etopox[4:]
Etopoy=Etopoy[4:]
Etopoelv=Etopoelv[4:]

ce=[Etopox,Etopoy,Etopoelv,r'$E-E^\prime$']

lE=[['cp_E','cp_E','green'],
    ['e1e2_E','e1e2_E','brown'],
    ['e2e3_E','e2e3_E','pink'],
    ['e3eO_E','e3eO_E','#D19A6B'],
    ['om_E','om_E','yellow'],
    ['pal_E','pal_E','black'],
    ['pep_E','pep_E','darkviolet']]
    
corte_E=[get_csv(x) for x in lE]

corte_E[4][0]=[corte_E[4][0][0][:93],corte_E[4][0][1][:93],corte_E[4][0][2][:93]]
corte_E[6][0]=[corte_E[6][0][0][3:],corte_E[6][0][1][3:],corte_E[6][0][2][3:]]
corte_E[1][0]=[corte_E[1][0][0][:99],corte_E[1][0][1][:99],corte_E[1][0][2][:99]]
corte_E[2][0]=[corte_E[2][0][0][:79],corte_E[2][0][1][:79],corte_E[2][0][2][:79]]
corte_E[3][0]=[corte_E[3][0][0][:57],corte_E[3][0][1][:57],corte_E[3][0][2][:57]]

c_E=plane_data(corte_E,'E')+[go.Scatter3d(x=ce[0], y=ce[1], z=ce[2],
            mode ='lines',
            name='Cross-section E',
            legendgroup='E',#'E',
            showlegend=True,
            line=dict(color='black',
                      width=6)
                        )]

# contacts names

cp_E=corte_E[0][0]
e1e2_E=corte_E[1][0]
e2e3_E=corte_E[2][0]

e3eO_E=corte_E[3][0]
om_E=corte_E[4][0]

pal_E=corte_E[5][0]

pep_E=corte_E[6][0]

pEO=[om_E[0][::-1]+pal_E[0][27:]+[om_E[0][-1]],
    om_E[1][::-1]+pal_E[1][27:]+[om_E[1][-1]],
    om_E[2][::-1]+pal_E[2][27:]+[300]]
pEM=[Etopox[:22]+pal_E[0][:18]+om_E[0],
    Etopoy[:22]+pal_E[1][:18]+om_E[1],
    Etopoelv[:22]+pal_E[2][:18]+om_E[2]]
pEC=[pal_E[0][27:][::-1]+cp_E[0],
    pal_E[1][27:][::-1]+cp_E[1],
    pal_E[2][27:][::-1]+cp_E[2]]
pEP=[pal_E[0][:27][::-1]+Etopox[21:23]+pep_E[0]+cp_E[0][::-1],
    pal_E[1][:27][::-1]+Etopoy[21:23]+pep_E[1]+cp_E[1][::-1],
    pal_E[2][:27][::-1]+Etopoelv[21:23]+pep_E[2]+cp_E[2][::-1]]
pEE1=[pep_E[0][::-1]+Etopox[23:37]+e1e2_E[0]+[e1e2_E[0][-1]],
     pep_E[1][::-1]+Etopoy[23:37]+e1e2_E[1]+[e1e2_E[1][-1]],
     pep_E[2][::-1]+Etopoelv[23:37]+e1e2_E[2]+[300]]
pEE2=[e1e2_E[0][::-1]+Etopox[39:48]+e2e3_E[0],
     e1e2_E[1][::-1]+Etopoy[39:48]+e2e3_E[1],
     e1e2_E[2][::-1]+Etopoelv[39:48]+e2e3_E[2]]
pEE3=[e2e3_E[0][::-1]+Etopox[49:63]+e3eO_E[0],
     e2e3_E[1][::-1]+Etopoy[49:63]+e3eO_E[1],
     e2e3_E[2][::-1]+Etopoelv[49:63]+e3eO_E[2]]
pEE4=[e3eO_E[0][::-1]+Etopox[63:],
     e3eO_E[1][::-1]+Etopoy[63:],
     e3eO_E[2][::-1]+Etopoelv[63:]]

EO=fil_pol(pEO,gE)
EM=fil_pol(pEM,gE)
EC=fil_pol(pEC,gE)
EP=fil_pol(pEP,gE)
EE1=fil_pol(pEE1,gE)
EE2=fil_pol(pEE2,gE)
EE3=fil_pol(pEE3,gE)
EE4=fil_pol(pEE4,gE)

pEO_dat=[go.Scatter3d(x=pEO[0], y=pEO[1], z=pEO[2],
            mode ='lines',
            name='O2E',
            legendgroup='E',#'O2E',
            showlegend=False,
            line=dict(color='#E19A6B',
                      width=8)
                        )]
EO_dat=[go.Scatter3d(x=EO[0],y=EO[1],z=EO[2],
                     mode ='markers',
                     name='O2E',
                     legendgroup='E',#'O2E',
                     showlegend=False,
                     marker=dict(color='#E19A6B',
                     size=3))]


pEM_dat=[go.Scatter3d(x=pEM[0], y=pEM[1], z=pEM[2],
            mode ='lines',
            name='M2E',
            legendgroup='E',#'M2E',
            showlegend=False,
            line=dict(color='yellow',
                      width=8)
                        )]
EM_dat=[go.Scatter3d(x=EM[0],y=EM[1],z=EM[2],
                     mode ='markers',
                     name='M2E',
                     legendgroup='E',#'M2E',
                     showlegend=False,
                     marker=dict(color='yellow',
                     size=3))]

pEC_dat=[go.Scatter3d(x=pEC[0], y=pEC[1], z=pEC[2],
            mode ='lines',
            name='C2E',
            legendgroup='E',#'C2E',
            showlegend=False,
            line=dict(color='#E7F4E2',
                      width=8)
                        )]
EC_dat=[go.Scatter3d(x=EC[0],y=EC[1],z=EC[2],
                     mode ='markers',
                     name='C2E',
                     legendgroup='E',#'C2E',
                     showlegend=False,
                     marker=dict(color='#E7F4E2',
                     size=3))]


pEP_dat=[go.Scatter3d(x=pEP[0], y=pEP[1], z=pEP[2],
            mode ='lines',
            name='PE',
            legendgroup='E',#'PE',
            showlegend=False,
            line=dict(color='#e6d7ff',
                      width=8)
                        )]
EP_dat=[go.Scatter3d(x=EP[0],y=EP[1],z=EP[2],
                     mode ='markers',
                     name='PE',
                     legendgroup='E',#'PE',
                     showlegend=False,
                     marker=dict(color='#e6d7ff',
                     size=3))]
pEE1_dat=[go.Scatter3d(x=pEE1[0], y=pEE1[1], z=pEE1[2],
            mode ='lines',
            name='E1D',
            legendgroup='E',#'E1D',
            showlegend=False,
            line=dict(color='#E97451',
                      width=8)
                        )]
EE1_dat=[go.Scatter3d(x=EE1[0],y=EE1[1],z=EE1[2],
                     mode ='markers',
                     name='E1D',
                     legendgroup='E',#'E1D',
                     showlegend=False,
                     marker=dict(color='#E97451',
                     size=3))]


pEE2_dat=[go.Scatter3d(x=pEE2[0], y=pEE2[1], z=pEE2[2],
            mode ='lines',
            name='E2D',
            legendgroup='E',#'E2D',
            showlegend=False,
            line=dict(color='pink',
                      width=8)
                        )]
EE2_dat=[go.Scatter3d(x=EE2[0],y=EE2[1],z=EE2[2],
                     mode ='markers',
                     name='E2D',
                     legendgroup='E',#'E2D',
                     showlegend=False,
                     marker=dict(color='pink',
                     size=3))]


pEE3_dat=[go.Scatter3d(x=pEE3[0], y=pEE3[1], z=pEE3[2],
            mode ='lines',
            name='E3E',
            legendgroup='E',#'E3E',
            showlegend=False,
            line=dict(color='#E1BEA8',
                      width=8)
                        )]
EE3_dat=[go.Scatter3d(x=EE3[0],y=EE3[1],z=EE3[2],
                     mode ='markers',
                     name='E3E',
                     legendgroup='E',#'E3E',
                     showlegend=False,
                     marker=dict(color='#E1BEA8',
                     size=3))]
pEE4_dat=[go.Scatter3d(x=pEE4[0], y=pEE4[1], z=pEE4[2],
            mode ='lines',
            name='E4E',
            legendgroup='E',#'E4E',
            showlegend=False,
            line=dict(color='#E19A6B',
                      width=8)
                        )]
EE4_dat=[go.Scatter3d(x=EE4[0],y=EE4[1],z=EE4[2],
                     mode ='markers',
                     name='E4E',
                     legendgroup='E',#'E4E',
                     showlegend=False,
                     marker=dict(color='#E19A6B',
                     size=3))]



In [ ]:
se_dat=pEO_dat+EO_dat+pEM_dat+EM_dat+pEC_dat+EC_dat+pEP_dat+EP_dat+pEE1_dat+EE1_dat+pEE2_dat+EE2_dat+pEE3_dat+EE3_dat+pEE4_dat+EE4_dat+c_E

In [ ]:
nx=[secx[0]+130]+[secx[2*i]+280 for i in range(1,len(secx)//2)]+[secx[2*i+1]+100 for i in range((len(secx)//2)-1)
                                                ]+[secx[-1]-150]
ny=[secy[0]-200]+[secy[2*i]-500 for i in range(1,len(secy)//2)]+[secy[2*i+1]-100 for i in range((len(secy)//2)-1)
                                                ]+[secy[-1]+350]
nz=[750,700,700,700,600,700,700,700,600,700]
ntxt=['A','B','C','D','E','A´','B´','D´','C´','E´']

In [ ]:
tsa=[go.Scatter3d(x=[nx[0],nx[5]], y=[ny[0],ny[5]], z=[nz[0],nz[5]],
                   text=[ntxt[0],ntxt[5]],
                   name='Section A',
                   mode="text",
                   legendgroup='A',
                   showlegend=False,
                   textfont=dict(color="orange",size=18),
                   hoverinfo="skip")]

sa_datt=sa_dat+tsa

tsb=[go.Scatter3d(x=[nx[1],nx[6]], y=[ny[1],ny[6]], z=[nz[1],nz[6]],
                   text=[ntxt[1],ntxt[6]],
                   name='Section B',
                   mode="text",
                   legendgroup='B',
                   showlegend=False,
                   textfont=dict(color="orange",size=18),
                   hoverinfo="skip")]

sb_datt=sb_dat+tsb

tsc=[go.Scatter3d(x=[nx[2],nx[7]], y=[ny[2],ny[7]], z=[nz[2],nz[7]],
                   text=[ntxt[2],ntxt[7]],
                   name='Section C',
                   mode="text",
                   legendgroup='C',
                   showlegend=False,
                   textfont=dict(color="orange",size=18),
                   hoverinfo="skip")]

sc_datt=sc_dat+tsc

tsd=[go.Scatter3d(x=[nx[3],nx[8]], y=[ny[3],ny[8]], z=[nz[3],nz[8]],
                   text=[ntxt[3],ntxt[8]],
                   name='Section D',
                   mode="text",
                   legendgroup='D',
                   showlegend=False,
                   textfont=dict(color="orange",size=18),
                   hoverinfo="skip")]

sd_datt=sd_dat+tsd

tse=[go.Scatter3d(x=[nx[4],nx[9]], y=[ny[4],ny[9]], z=[nz[4],nz[9]],
                   text=[ntxt[4],ntxt[9]],
                   name='Section E',
                   mode="text",
                   legendgroup='E',
                   showlegend=False,
                   textfont=dict(color="orange",size=18),
                   hoverinfo="skip")]

se_datt=se_dat+tse

We create the 3D model with real topography

In [ ]:
fig=go.Figure(contacts+cabs+faults+tcont+sa_datt+sb_datt+sc_datt+sd_datt+se_datt)

# topography

fig.add_trace(go.Surface(  x=topo_linear[3],y=topo_linear[4],z=topo_linear[5],
                           colorscale='brwnyl',
                           opacity = 1,
                           name='Topography',
                           legendgroup='topo',
                           showlegend=True,
                           showscale=False))
fig.add_trace(go.Surface(  x=topo_linear[3],y=topo_linear[4],z=topo_linear[5],
                           colorscale='gray',
                           opacity = 0.3,
                           name='Topo',
                           legendgroup='topo',
                           showlegend=False,
                           showscale=False))




#bordes

fig.add_trace(go.Scatter3d(x=xl1,
                        y=yl1,
                        z=elvl1,
                       mode ='lines',
                       name='Border',
                      legendgroup='border',
                      showlegend=True,
                      line=dict(color='black',
                      width=3)
                       ))

fig.add_trace(go.Scatter3d(x=xl2,
                        y=yl2,
                        z=elvl2,
                       mode ='lines',
                       name='sur',
                      legendgroup='border',
                      showlegend=False,
                      line=dict(color='black',
                      width=3)
                       ))

fig.add_trace(go.Scatter3d(x=xl3,
                        y=yl3,
                        z=elvl3,
                       mode ='lines',
                       name='norte',
                      legendgroup='border',
                      showlegend=False,
                      line=dict(color='black',
                      width=3)
                       ))
fig.add_trace(go.Scatter3d(x=xl4,
                        y=yl4,
                        z=elvl4,
                       mode ='lines',
                       name='este',
                      legendgroup='border',
                      showlegend=False,
                      line=dict(color='black',
                      width=3)
                       ))



# texts

trace = go.Scatter3d(
                   x=namesx, y=namesy, z=namesz,
                   text=namestxt,
                   name='Heights',
                   mode="text",
                   textfont=dict(color=["black","black"],size=13),
                   hoverinfo="skip")

fig.add_trace(trace)

trace2 = go.Scatter3d(
                   x=nnx, y=nny, z=nnz,
                   text=nntxt,
                   name='Stratigraphic levels',
                   mode="text",
                   legendgroup='cabalgamientos',
                   showlegend=False,
                   textfont=dict(color='black',size=18),
                   hoverinfo="skip")

fig.add_trace(trace2)
 
# geological symbos for faults

fig.add_trace(go.Scatter3d(x=f1[0],
                        y=f1[1],
                        z=f1[2],
                       mode ='lines',
                       name='Sections',
                      legendgroup='faults',
                      showlegend=False,
                      line=dict(color='black',
                      width=5)
                       ))
fig.add_trace(go.Scatter3d(x=f2[0],
                        y=f2[1],
                        z=f2[2],
                       mode ='lines',
                       name='Sections',
                      legendgroup='faults',
                      showlegend=False,
                      line=dict(color='black',
                      width=5)
                       ))

# geological symbos for thrusts

fig.add_trace(go.Scatter3d(x=[pal_B[0][55],pal_B[0][50],pal_B[0][65]],
                        y=[pal_B[1][55],pal_B[1][50],pal_B[1][65]],
                        z=[pal_B[2][55]+50,pal_B[2][50]+30,pal_B[2][65]+30],
                       mode ='lines',
                       name='Sections',
                      legendgroup='B',
                      showlegend=False,
                      line=dict(color='black',
                      width=5)
                       ))
fig.add_trace(go.Scatter3d(x=[cal_B[0][55],cal_B[0][50],cal_B[0][65]],
                        y=[cal_B[1][55],cal_B[1][50],cal_B[1][65]],
                        z=[cal_B[2][55]+50,cal_B[2][50]+30,cal_B[2][65]+30],
                       mode ='lines',
                       name='Sections',
                      legendgroup='B',
                      showlegend=False,
                      line=dict(color='black',
                      width=5)
                       ))


fig.add_trace(go.Scatter3d(x=[pal_C[0][55],pal_C[0][50],pal_C[0][65]],
                        y=[pal_C[1][55],pal_C[1][50],pal_C[1][65]],
                        z=[pal_C[2][55]+50,pal_C[2][50]+30,pal_C[2][65]+30],
                       mode ='lines',
                       name='Sections',
                      legendgroup='C',
                      showlegend=False,
                      line=dict(color='black',
                      width=5)
                       ))

fig.add_trace(go.Scatter3d(x=[cal_C[0][55],cal_C[0][50],cal_C[0][65]],
                        y=[cal_C[1][55],cal_C[1][50],cal_C[1][65]],
                        z=[cal_C[2][55]+50,cal_C[2][50]+30,cal_C[2][65]+30],
                       mode ='lines',
                       name='Sections',
                      legendgroup='C',
                      showlegend=False,
                      line=dict(color='black',
                      width=5)
                       ))

fig.add_trace(go.Scatter3d(x=[pal_D[0][55],pal_D[0][50],pal_D[0][65]],
                        y=[pal_D[1][55],pal_D[1][50],pal_D[1][65]],
                        z=[pal_D[2][55]+50,pal_D[2][50]+30,pal_D[2][65]+30],
                       mode ='lines',
                       name='Sections',
                      legendgroup='D',
                      showlegend=False,
                      line=dict(color='black',
                      width=5)
                       ))

fig.add_trace(go.Scatter3d(x=[pal_E[0][55],pal_E[0][50],pal_E[0][65]],
                        y=[pal_E[1][55],pal_E[1][50],pal_E[1][65]],
                        z=[pal_E[2][55]+50,pal_E[2][50]+30,pal_E[2][65]+30],
                       mode ='lines',
                       name='Sections',
                      legendgroup='E',
                      showlegend=False,
                      line=dict(color='black',
                      width=5)
                       ))

fig.add_trace(go.Scatter3d(x=[pal_ds_B[0][55],pal_ds_B[0][50],pal_ds_B[0][65]],
                        y=[pal_ds_B[1][55],pal_ds_B[1][50],pal_ds_B[1][65]],
                        z=[pal_ds_B[2][55]+50,pal_ds_B[2][50]+30,pal_ds_B[2][65]+30],
                       mode ='lines',
                       name='Sections',
                      legendgroup='B',
                      showlegend=False,
                      line=dict(color='black',
                      width=5)
                       ))
fig.add_trace(go.Scatter3d(x=[calds_A[0][55],calds_A[0][50],calds_A[0][65]],
                        y=[calds_A[1][55],calds_A[1][50],calds_A[1][65]],
                        z=[calds_A[2][55]+50,calds_A[2][50]+30,calds_A[2][65]+30],
                       mode ='lines',
                       name='Sections',
                      legendgroup='A',
                      showlegend=False,
                      line=dict(color='black',
                      width=5)
                       ))



# figure layout

camera = dict(up=dict(x=0, y=0, z=1),
              center=dict(x=0, y=0, z=0),
               eye=dict(x=0, y=-1, z=2) )

fig.update_layout( title="3D Palomeque sheets geological map with cross-sections",
                  #paper_bgcolor = 'black',
                 scene = dict(
                     xaxis=dict(title='UTM_X', 
                                tickfont = dict(size = 10,color = 'black'),
                                title_font_size=10,
                                titlefont_color='black', 
                                range=[cotasxy[0],cotasxy[1]],
                                backgroundcolor='white',
                                color='black',
                                gridcolor='gray'),
                     yaxis=dict(title='UTM_Y',
                                tickfont = dict(size = 10,color = 'black'),
                                title_font_size=10,
                                titlefont_color='black', 
                                range=[cotasxy[2],cotasxy[3]],  
                                backgroundcolor='white',
                                color='black',
                                gridcolor='gray'),
                      zaxis=dict(nticks=4,
                                tickfont = dict(size = 10,color = 'black'),
                                title='Elevation', 
                                title_font_size=10,
                                titlefont_color='black', 
                                range=[100,1500],
                                backgroundcolor='white',
                                color='black', 
                                gridcolor='gray'),
                     aspectratio=dict(x=2, y=1.8, z=1)),
                     #aspectmode='data'),
                     scene_camera= camera
                 ) 
              

fig.show()
go_offline.plot(fig,filename=FIGURESDIR+'3d.html',validate=True, auto_open=False)


Now we change the colors of the surface

In [ ]:
fig=go.Figure(contacts+cabs+faults+tcont+sa_datt+sb_datt+sc_datt+sd_datt+se_datt+p_dat)

#bordes

fig.add_trace(go.Scatter3d(x=xl1,
                        y=yl1,
                        z=elvl1,
                       mode ='lines',
                       name='Border',
                      legendgroup='border',
                      showlegend=True,
                      line=dict(color='black',
                      width=3)
                       ))

fig.add_trace(go.Scatter3d(x=xl2,
                        y=yl2,
                        z=elvl2,
                       mode ='lines',
                       name='sur',
                      legendgroup='border',
                      showlegend=False,
                      line=dict(color='black',
                      width=3)
                       ))

fig.add_trace(go.Scatter3d(x=xl3,
                        y=yl3,
                        z=elvl3,
                       mode ='lines',
                       name='norte',
                      legendgroup='border',
                      showlegend=False,
                      line=dict(color='black',
                      width=3)
                       ))
fig.add_trace(go.Scatter3d(x=xl4,
                        y=yl4,
                        z=elvl4,
                       mode ='lines',
                       name='este',
                      legendgroup='border',
                      showlegend=False,
                      line=dict(color='black',
                      width=3)
                       ))



# texts

trace = go.Scatter3d(
                   x=namesx, y=namesy, z=namesz,
                   text=namestxt,
                   name='Heights',
                   mode="text",
                   textfont=dict(color=["black","black"],size=13),
                   hoverinfo="skip")

fig.add_trace(trace)

trace2 = go.Scatter3d(
                   x=nnx, y=nny, z=nnz,
                   text=nntxt,
                   name='Stratigraphic levels',
                   mode="text",
                   legendgroup='cabalgamientos',
                   showlegend=False,
                   textfont=dict(color='black',size=18),
                   hoverinfo="skip")

fig.add_trace(trace2)
 
# geological symbos for faults

fig.add_trace(go.Scatter3d(x=f1[0],
                        y=f1[1],
                        z=f1[2],
                       mode ='lines',
                       name='Sections',
                      legendgroup='faults',
                      showlegend=False,
                      line=dict(color='black',
                      width=5)
                       ))
fig.add_trace(go.Scatter3d(x=f2[0],
                        y=f2[1],
                        z=f2[2],
                       mode ='lines',
                       name='Sections',
                      legendgroup='faults',
                      showlegend=False,
                      line=dict(color='black',
                      width=5)
                       ))

# geological symbos for thrusts

fig.add_trace(go.Scatter3d(x=[pal_B[0][55],pal_B[0][50],pal_B[0][65]],
                        y=[pal_B[1][55],pal_B[1][50],pal_B[1][65]],
                        z=[pal_B[2][55]+50,pal_B[2][50]+30,pal_B[2][65]+30],
                       mode ='lines',
                       name='Sections',
                      legendgroup='B',
                      showlegend=False,
                      line=dict(color='black',
                      width=5)
                       ))
fig.add_trace(go.Scatter3d(x=[cal_B[0][55],cal_B[0][50],cal_B[0][65]],
                        y=[cal_B[1][55],cal_B[1][50],cal_B[1][65]],
                        z=[cal_B[2][55]+50,cal_B[2][50]+30,cal_B[2][65]+30],
                       mode ='lines',
                       name='Sections',
                      legendgroup='B',
                      showlegend=False,
                      line=dict(color='black',
                      width=5)
                       ))


fig.add_trace(go.Scatter3d(x=[pal_C[0][55],pal_C[0][50],pal_C[0][65]],
                        y=[pal_C[1][55],pal_C[1][50],pal_C[1][65]],
                        z=[pal_C[2][55]+50,pal_C[2][50]+30,pal_C[2][65]+30],
                       mode ='lines',
                       name='Sections',
                      legendgroup='C',
                      showlegend=False,
                      line=dict(color='black',
                      width=5)
                       ))

fig.add_trace(go.Scatter3d(x=[cal_C[0][55],cal_C[0][50],cal_C[0][65]],
                        y=[cal_C[1][55],cal_C[1][50],cal_C[1][65]],
                        z=[cal_C[2][55]+50,cal_C[2][50]+30,cal_C[2][65]+30],
                       mode ='lines',
                       name='Sections',
                      legendgroup='C',
                      showlegend=False,
                      line=dict(color='black',
                      width=5)
                       ))

fig.add_trace(go.Scatter3d(x=[pal_D[0][55],pal_D[0][50],pal_D[0][65]],
                        y=[pal_D[1][55],pal_D[1][50],pal_D[1][65]],
                        z=[pal_D[2][55]+50,pal_D[2][50]+30,pal_D[2][65]+30],
                       mode ='lines',
                       name='Sections',
                      legendgroup='D',
                      showlegend=False,
                      line=dict(color='black',
                      width=5)
                       ))

fig.add_trace(go.Scatter3d(x=[pal_E[0][55],pal_E[0][50],pal_E[0][65]],
                        y=[pal_E[1][55],pal_E[1][50],pal_E[1][65]],
                        z=[pal_E[2][55]+50,pal_E[2][50]+30,pal_E[2][65]+30],
                       mode ='lines',
                       name='Sections',
                      legendgroup='E',
                      showlegend=False,
                      line=dict(color='black',
                      width=5)
                       ))

fig.add_trace(go.Scatter3d(x=[pal_ds_B[0][55],pal_ds_B[0][50],pal_ds_B[0][65]],
                        y=[pal_ds_B[1][55],pal_ds_B[1][50],pal_ds_B[1][65]],
                        z=[pal_ds_B[2][55]+50,pal_ds_B[2][50]+30,pal_ds_B[2][65]+30],
                       mode ='lines',
                       name='Sections',
                      legendgroup='B',
                      showlegend=False,
                      line=dict(color='black',
                      width=5)
                       ))
fig.add_trace(go.Scatter3d(x=[calds_A[0][55],calds_A[0][50],calds_A[0][65]],
                        y=[calds_A[1][55],calds_A[1][50],calds_A[1][65]],
                        z=[calds_A[2][55]+50,calds_A[2][50]+30,calds_A[2][65]+30],
                       mode ='lines',
                       name='Sections',
                      legendgroup='A',
                      showlegend=False,
                      line=dict(color='black',
                      width=5)
                       ))



# figure layout

camera = dict(up=dict(x=0, y=0, z=1),
              center=dict(x=0, y=0, z=0),
               eye=dict(x=0, y=-1, z=2) )

fig.update_layout( title="3D Palomeque sheets geological map with cross-sections",
                  #paper_bgcolor = 'black',
                 scene = dict(
                     xaxis=dict(title='UTM_X', 
                                tickfont = dict(size = 10,color = 'black'),
                                title_font_size=10,
                                titlefont_color='black', 
                                range=[cotasxy[0],cotasxy[1]],
                                backgroundcolor='white',
                                color='black',
                                gridcolor='gray'),
                     yaxis=dict(title='UTM_Y',
                                tickfont = dict(size = 10,color = 'black'),
                                title_font_size=10,
                                titlefont_color='black', 
                                range=[cotasxy[2],cotasxy[3]],  
                                backgroundcolor='white',
                                color='black',
                                gridcolor='gray'),
                      zaxis=dict(nticks=4,
                                tickfont = dict(size = 10,color = 'black'),
                                title='Elevation', 
                                title_font_size=10,
                                titlefont_color='black', 
                                range=[100,1500],
                                backgroundcolor='white',
                                color='black', 
                                gridcolor='gray'),
                     aspectratio=dict(x=2, y=1.8, z=1)),
                     #aspectmode='data'),
                     scene_camera= camera
                 ) 
              

fig.show()
go_offline.plot(fig,filename=FIGURESDIR+'3dandxscolors.html',validate=True, auto_open=False)
